In [ ]:
import os as os

RELATIVE = os.path.join('F:', 'Mauro', 'InceptionResNetV2')

In [ ]:
from os.path import join
import tensorflow.keras.optimizers as tfko
import imgaug.augmenters as iaa
import tensorflow.keras.callbacks as tfkc

from utils.data_load import load
from utils.data_transform import transform
from utils.data_augmentation import DataGenerator

from os.path import exists
from os import mkdir
from tensorflow.keras.utils import to_categorical
from imgaug import ALL
from numpy import sqrt


RELATIVE = join('F:/', 'Mauro', 'InceptionResNetV2')


N = 1200

TEST_PERCENTAGE = 1/6

train_images, train_labels, test_images, test_labels = load(
    RELATIVE, N, TEST_PERCENTAGE)


CLASS_NAMES = ['neutral', 'smoke']

IMAGE_DIMENSIONS = (512, 512, 3)


EPOCHS = 200

BATCH_SIZE = 1

LEARNING_RATE = 0.001

LOSS = 'categorical_crossentropy'
OPTIMIZER = tfko.Adam(learning_rate=LEARNING_RATE)

NUM_CLASSES = len(CLASS_NAMES)

if LOSS == 'categorical_crossentropy':
    NUM_CLASSES = len(CLASS_NAMES)
    train_labels = to_categorical(train_labels, NUM_CLASSES)
    test_labels = to_categorical(test_labels, NUM_CLASSES)


def augmenters():

    def sometimes(aug): return iaa.Sometimes(0.5, aug)

    return iaa.Sequential(
        [
            iaa.Fliplr(0.5),
            iaa.Flipud(0.1),
            sometimes(
                iaa.Affine(scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
                           translate_percent={
                               "x": (-0.1, 0.1), "y": (-0.1, 0.1)},
                           order=[0, 1],
                           cval=(0, 255),
                           mode=ALL)),

        ],
        random_order=True)


CHECKPOINTS_DIR = join(RELATIVE, 'checkpoints')
if not exists(CHECKPOINTS_DIR):
    mkdir(CHECKPOINTS_DIR)


def lr_schedule(epoch):
    learning_rate = 0.001
    if epoch > 180:
        learning_rate *= 0.0005
    elif epoch > 160:
        learning_rate *= 0.001
    elif epoch > 120:
        learning_rate *= 0.01
    elif epoch > 80:
        learning_rate *= 0.1
    return learning_rate


callbacks = [
    tfkc.ModelCheckpoint(filepath=join(CHECKPOINTS_DIR, 'checkpoint{epoch:04d}.hdf5'),
                         monitor='val_loss',
                         save_weights_only=True,
                         save_best_only=True,
                         verbose=1),
    tfkc.LearningRateScheduler(lr_schedule),
    tfkc.ReduceLROnPlateau(factor=sqrt(0.1),
                           patience=5,
                           min_lr=0.0000005,
                           verbose=1)
]


train_data = DataGenerator(train_images,
                           train_labels,
                           batch_size=BATCH_SIZE,
                           image_dimensions=IMAGE_DIMENSIONS,
                           transform=transform(IMAGE_DIMENSIONS[:-1]),
                           augmenters=augmenters(),
                           shuffle=True)

test_data = DataGenerator(test_images,
                          test_labels,
                          batch_size=BATCH_SIZE,
                          image_dimensions=IMAGE_DIMENSIONS,
                          transform=transform(IMAGE_DIMENSIONS[:-1]))


In [ ]:
train_data[0][0]

In [ ]:
seq = []
seq.append(ia.augmenters.size.CenterPadToAspectRatio(1.0,
                                                     pad_mode='edge',
                                                     pad_cval=2))
seq.append(ia.augmenters.size.Resize((256, 256),
                                     interpolation=cv.INTER_AREA))
augmenters = ia.augmenters.Sequential(seq)

test_images_cv = []
for fn in test_images:
    test_images_cv.append(cv.cvtColor(cv.imread(str(fn[0])), cv.COLOR_BGR2RGB))

final_aug = augmenters.augment_images(test_images_cv)
final_aug = np.array([img/255.0 for img in final_aug])

In [ ]:
test_data[3][0][0]

In [ ]:
final_aug[3]

In [ ]:
import matplotlib.pyplot as plt

train_data = DataGenerator(train_images,
                           train_labels,
                           batch_size=BATCH_SIZE,
                           image_dimensions=IMAGE_DIMENSIONS,
                           transform=transform,
                           # augmenters=augmenters,
                           shuffle=True)

plt.figure(figsize=(25,25))
for i in range(32):
    plt.subplot(8,4, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_data[i][0][0])
    plt.xlabel(str(train_data[i][1][0][1]))
plt.show()

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=IMAGE_DIMENSIONS))

model.add(tf.keras.layers.Conv2D(32,
                                 kernel_size=(3, 3),
                                 activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64,
                                 (3, 3),
                                 activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64,
                                 kernel_size=(3, 3),
                                 activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(128,
                                 (3, 3),
                                 activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(128,
                                 kernel_size=(3, 3),
                                 activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128,
                                activation='relu'))
model.add(tf.keras.layers.Dense(1,
                                activation='sigmoid'))
model.summary()

model.compile(loss=LOSS,
              optimizer=OPTIMIZER,
              metrics=['accuracy'])

In [ ]:
import six as sx
import math as mt
import string as st
import keras_applications as ka
import collections as co
import functools as fu


DENSE_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 1. / 3.,
        'mode': 'fan_out',
        'distribution': 'uniform'
    }
}


def get_submodules_from_kwargs(kwargs):
    backend = kwargs.get('backend', _KERAS_BACKEND)
    layers = kwargs.get('layers', _KERAS_LAYERS)
    models = kwargs.get('models', _KERAS_MODELS)
    utils = kwargs.get('utils', _KERAS_UTILS)
    for key in kwargs.keys():
        if key not in ['backend', 'layers', 'models', 'utils']:
            raise TypeError('Invalid keyword argument: %s', key)
    return backend, layers, models, utils


def inject_tfkeras_modules(func):
    @fu.wraps(func)
    def wrapper(*args, **kwargs):
        kwargs['backend'] = tf.keras.backend
        kwargs['layers'] = tf.keras.layers
        kwargs['models'] = tf.keras.models
        kwargs['utils'] = tf.keras.utils
        return func(*args, **kwargs)
    return wrapper


def get_swish(**kwargs):
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    def swish(x):
        return x * backend.sigmoid(x)

    return swish


def get_dropout(**kwargs):
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    class FixedDropout(layers.Dropout):
        def _get_noise_shape(self, inputs):
            if self.noise_shape is None:
                return self.noise_shape

            symbolic_shape = backend.shape(inputs)
            noise_shape = [symbolic_shape[axis] if shape is None else shape
                           for axis, shape in enumerate(self.noise_shape)]
            return tuple(noise_shape)

    return FixedDropout


def init_tfkeras_custom_objects():
    custom_objects = {
        'swish': inject_tfkeras_modules(get_swish)(),
        'FixedDropout': inject_tfkeras_modules(get_dropout)()
    }
    tf.keras.utils.get_custom_objects().update(custom_objects)


_KERAS_BACKEND = None
_KERAS_LAYERS = None
_KERAS_MODELS = None
_KERAS_UTILS = None


backend = None
layers = None
models = None
keras_utils = None

# *****************************************************************************************


def round_filters(filters, width_coefficient, depth_divisor):
    filters *= width_coefficient
    new_filters = int(filters + depth_divisor /
                      2) // depth_divisor * depth_divisor
    new_filters = max(depth_divisor, new_filters)
    if new_filters < 0.9 * filters:
        new_filters += depth_divisor
    return int(new_filters)


CONV_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 2.0,
        'mode': 'fan_out',
        'distribution': 'normal'
    }
}

BlockArgs = collections.namedtuple('BlockArgs', [
    'kernel_size', 'num_repeat', 'input_filters', 'output_filters',
    'expand_ratio', 'id_skip', 'strides', 'se_ratio'
])

DEFAULT_BLOCKS_ARGS = [
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16,
              expand_ratio=1, id_skip=True, strides=[1, 1], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=2, input_filters=16, output_filters=24,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=2, input_filters=24, output_filters=40,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=3, input_filters=40, output_filters=80,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=3, input_filters=80, output_filters=112,
              expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=4, input_filters=112, output_filters=192,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=192, output_filters=320,
              expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25)
]


def mb_conv_block(inputs, block_args, activation, drop_rate=None, prefix='', ):

    has_se = (block_args.se_ratio is not None) and (
        0 < block_args.se_ratio <= 1)
    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

    Dropout = get_dropout(
        backend=backend,
        layers=layers,
        models=models,
        utils=keras_utils
    )

    filters = block_args.input_filters * block_args.expand_ratio
    if block_args.expand_ratio != 1:
        x = layers.Conv2D(filters, 1,
                          padding='same',
                          use_bias=False,
                          kernel_initializer=CONV_KERNEL_INITIALIZER,
                          name=prefix + 'expand_conv')(inputs)
        x = layers.BatchNormalization(
            axis=bn_axis, name=prefix + 'expand_bn')(x)
        x = layers.Activation(activation, name=prefix + 'expand_activation')(x)
    else:
        x = inputs

    x = layers.DepthwiseConv2D(block_args.kernel_size,
                               strides=block_args.strides,
                               padding='same',
                               use_bias=False,
                               depthwise_initializer=CONV_KERNEL_INITIALIZER,
                               name=prefix + 'dwconv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=prefix + 'bn')(x)
    x = layers.Activation(activation, name=prefix + 'activation')(x)

    if has_se:
        num_reduced_filters = max(1, int(
            block_args.input_filters * block_args.se_ratio
        ))
        se_tensor = layers.GlobalAveragePooling2D(
            name=prefix + 'se_squeeze')(x)

        target_shape = (1, 1, filters) if backend.image_data_format(
        ) == 'channels_last' else (filters, 1, 1)
        se_tensor = layers.Reshape(
            target_shape, name=prefix + 'se_reshape')(se_tensor)
        se_tensor = layers.Conv2D(num_reduced_filters, 1,
                                  activation=activation,
                                  padding='same',
                                  use_bias=True,
                                  kernel_initializer=CONV_KERNEL_INITIALIZER,
                                  name=prefix + 'se_reduce')(se_tensor)
        se_tensor = layers.Conv2D(filters, 1,
                                  activation='sigmoid',
                                  padding='same',
                                  use_bias=True,
                                  kernel_initializer=CONV_KERNEL_INITIALIZER,
                                  name=prefix + 'se_expand')(se_tensor)
        if backend.backend() == 'theano':
            pattern = ([True, True, True, False] if backend.image_data_format() == 'channels_last'
                       else [True, False, True, True])
            se_tensor = layers.Lambda(
                lambda x: backend.pattern_broadcast(x, pattern),
                name=prefix + 'se_broadcast')(se_tensor)
        x = layers.multiply([x, se_tensor], name=prefix + 'se_excite')

    x = layers.Conv2D(block_args.output_filters, 1,
                      padding='same',
                      use_bias=False,
                      kernel_initializer=CONV_KERNEL_INITIALIZER,
                      name=prefix + 'project_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=prefix + 'project_bn')(x)
    if block_args.id_skip and all(
            s == 1 for s in block_args.strides
    ) and block_args.input_filters == block_args.output_filters:
        if drop_rate < 1 and (drop_rate > 0):
            x = Dropout(drop_rate,
                        noise_shape=(None, 1, 1, 1),
                        name=prefix + 'drop')(x)
        x = layers.add([x, inputs], name=prefix + 'add')

    return x


def EfficientNet(width_coefficient,
                 depth_coefficient,
                 default_resolution,
                 dropout_rate=0.2,
                 drop_connect_rate=0.2,
                 depth_divisor=8,
                 blocks_args=DEFAULT_BLOCKS_ARGS,
                 model_name='efficientnet',
                 include_top=True,
                 weights='imagenet',
                 input_tensor=None,
                 input_shape=None,
                 pooling=None,
                 classes=1000,
                 last_activation='softmax',
                 **kwargs):
    global backend, layers, models, keras_utils
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    if not (weights in {'imagenet', 'noisy-student', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as `"imagenet"` with `include_top`'
                         ' as true, `classes` should be 1000')

# *********************************************************************************************************************
    from keras_applications.imagenet_utils import _obtain_input_shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=default_resolution,
                                      min_size=32,
                                      data_format=backend.image_data_format(),
                                      require_flatten=include_top,
                                      weights=weights)

    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        from tensorflow.python.keras.backend import is_keras_tensor
        if not is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1
    activation = get_swish(**kwargs)

    x = img_input
    x = layers.Conv2D(round_filters(32, width_coefficient, depth_divisor), 3,
                      strides=(2, 2),
                      padding='same',
                      use_bias=False,
                      kernel_initializer=CONV_KERNEL_INITIALIZER,
                      name='stem_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='stem_bn')(x)
    x = layers.Activation(activation, name='stem_activation')(x)

    num_blocks_total = sum(block_args.num_repeat for block_args in blocks_args)
    block_num = 0
    for idx, block_args in enumerate(blocks_args):
        assert block_args.num_repeat > 0
        import math
        block_args = block_args._replace(
            input_filters=round_filters(block_args.input_filters,
                                        width_coefficient, depth_divisor),
            output_filters=round_filters(block_args.output_filters,
                                         width_coefficient, depth_divisor),
            num_repeat=int(math.ceil(depth_coefficient * block_args.num_repeat)))

        drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
        x = mb_conv_block(x, block_args,
                          activation=activation,
                          drop_rate=drop_rate,
                          prefix='block{}a_'.format(idx + 1))
        block_num += 1
        if block_args.num_repeat > 1:
            block_args = block_args._replace(
                input_filters=block_args.output_filters, strides=[1, 1])
            # ***************************************************************************************************
            from six.moves import xrange
            import string
            for bidx in xrange(block_args.num_repeat - 1):
                drop_rate = drop_connect_rate * \
                    float(block_num) / num_blocks_total
                block_prefix = 'block{}{}_'.format(
                    idx + 1,
                    string.ascii_lowercase[bidx + 1]
                )
                x = mb_conv_block(x, block_args,
                                  activation=activation,
                                  drop_rate=drop_rate,
                                  prefix=block_prefix)
                block_num += 1

    x = layers.Conv2D(round_filters(1280, width_coefficient, depth_divisor), 1,
                      padding='same',
                      use_bias=False,
                      kernel_initializer=CONV_KERNEL_INITIALIZER,
                      name='top_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='top_bn')(x)
    x = layers.Activation(activation, name='top_activation')(x)
    if include_top:
        x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        if dropout_rate and dropout_rate > 0:
            x = layers.Dropout(dropout_rate, name='top_dropout')(x)
        x = layers.Dense(classes,
                         activation=last_activation,
                         kernel_initializer=DENSE_KERNEL_INITIALIZER,
                         name='probs')(x)
    else:
        if pooling == 'avg':
            x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        elif pooling == 'max':
            x = layers.GlobalMaxPooling2D(name='max_pool')(x)

    if input_tensor is not None:
        inputs = keras_utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input

    model = models.Model(inputs, x, name=model_name)

    return model


def EfficientNetL2(
        include_top=True,
        weights=None,
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=1000,
        last_activation='softmax',
        **kwargs
):
    return EfficientNet(
        4.3, 5.3, 800, 0.5,
        model_name='efficientnet-l2',
        include_top=include_top, weights=weights,
        input_tensor=input_tensor, input_shape=input_shape,
        pooling=pooling, classes=classes, last_activation=last_activation,
        **kwargs
    )


setattr(EfficientNetL2, '__doc__', EfficientNet.__doc__)

EfficientNetL2 = inject_tfkeras_modules(EfficientNetL2)


def preprocess_input(x, **kwargs):
    kwargs = {k: v for k, v in kwargs.items() if k in [
        'backend', 'layers', 'models', 'utils']}
    return _preprocess_input(x, mode='torch', **kwargs)


preprocess_input = inject_tfkeras_modules(preprocess_input)

init_tfkeras_custom_objects()

model = EfficientNetL2(input_shape=IMAGE_DIMENSIONS,
                       classes=1,
                       last_activation='sigmoid')
model.summary()

model.compile(loss=LOSS,
              optimizer=OPTIMIZER,
              metrics=['accuracy'])

In [ ]:
model = tf.keras.applications.Xception(input_shape=IMAGE_DIMENSIONS,
                                       weights=None,
                                       classes=2)
model.summary()

model.compile(loss=LOSS,
              optimizer=OPTIMIZER,
              metrics=['accuracy'])

In [ ]:
callbacks = []
checkpoint_format = 'checkpoint{epoch:04d}_val-loss{loss:.4f}.hdf5'
callbacks.append(tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(CHECKPOINTS_DIR, checkpoint_format),
                                                    monitor='val_loss',
                                                    save_weights_only=True,
                                                    save_best_only=True))


def lr_schedule(epoch):
    learning_rate = 0.001
    if epoch > 180:
        learning_rate *= 0.0005
    elif epoch > 160:
        learning_rate *= 0.001
    elif epoch > 120:
        learning_rate *= 0.01
    elif epoch > 80:
        learning_rate *= 0.1
    return learning_rate


# callbacks.append(tf.keras.callbacks.LearningRateScheduler(lr_schedule))
callbacks.append(tf.keras.callbacks.ReduceLROnPlateau(factor=np.sqrt(0.1),
                                                      patience=5,
                                                      min_lr=0.0000005))

In [ ]:
history = model.fit_generator(generator=train_data,
                              validation_data=test_data,
                              epochs=EPOCHS,
                              callbacks=callbacks)

model.save(MODEL_DIR)

scores = model.evaluate(test_data,
                        verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
import datetime as dt
import six as sx
import math as mt
import string as st
import keras_applications as ka
import collections as co
import functools as fu
import tensorflow.python as tfp
import imgaug as ia
import sklearn.model_selection as skm
import tensorflow as tf
import random as rd
import numpy as np
import cv2 as cv
import glob as gl
import os as os

RELATIVE = os.path.join('F:', 'Mauro')


FILENAMES_NEUTRAL = list(
    gl.glob(os.path.join(RELATIVE, 'data', 'neutral', '*')))
FILENAMES_SMOKE = list(gl.glob(os.path.join(RELATIVE, 'data', 'smoke', '*')))
np.random.shuffle(FILENAMES_NEUTRAL)
np.random.shuffle(FILENAMES_SMOKE)

# ---------------------------------------------------------------------------------------
# ***************************************************************************************

MODEL_DIR = os.path.join(RELATIVE, 'model.h5')

CHECKPOINTS_DIR = os.path.join(RELATIVE, 'checkpoints')
if not os.path.exists(CHECKPOINTS_DIR):
    os.mkdir(CHECKPOINTS_DIR)

CLASS_NAMES = ['neutral', 'smoke']

N = 1200

TEST_PERCENTAGE = 1/6

IMAGE_DIMENSIONS = (128, 128, 3)

# ***************************************************************************************
# ---------------------------------------------------------------------------------------

DATA = []
for i in range(N):
    DATA.append([str(FILENAMES_NEUTRAL[i]), 0])
    DATA.append([str(FILENAMES_SMOKE[i]), 1])

DATA_SPLIT = np.hsplit(np.asarray(DATA), np.array([1, 2]))
DATA_IMAGES = DATA_SPLIT[0]
DATA_LABELS = DATA_SPLIT[1]
DATA_LABELS = DATA_LABELS.astype(np.int)

train_images, test_images, train_labels, test_labels = skm.train_test_split(DATA_IMAGES,
                                                                            DATA_LABELS,
                                                                            test_size=TEST_PERCENTAGE,
                                                                            shuffle=False)

In [ ]:
np.random.seed(seed=rd.randint(1, 999999))
s = np.arange(train_images.shape[0])
np.random.shuffle(s)
train_images = train_images[s]
train_labels = train_labels[s]

np.random.seed(seed=rd.randint(1, 999999))
s = np.arange(test_images.shape[0])
np.random.shuffle(s)
test_images = test_images[s]
test_labels = test_labels[s]

In [ ]:
n0=0
n1=0
for d in test_labels:
    if d[0]==0:
        n0+=1
    else:
        n1+=1
        
print(n0)
print(n1)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(25,25))
for i in range(32):
    plt.subplot(8,4, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(cv.cvtColor(cv.imread(str(test_images[i][0])), cv.COLOR_BGR2RGB))
    plt.xlabel(CLASS_NAMES[test_labels[i][0]])
plt.show()

In [ ]:
model.get_input_at(0)

In [ ]:
model.input.shape[1:]

In [ ]:

model.compile(loss=LOSS,
              optimizer=OPTIMIZER,
              metrics=['accuracy'])

history = model.fit_generator(generator=train_data,
                              validation_data=test_data,
                              epochs=EPOCHS,
                              callbacks=callbacks)

In [ ]:
import tensorflow as tf

model = tf.keras.applications.Xception(input_shape=(256, 256, 3),
                                       weights=None,
                                       classes=2)
model.load_weights(os.path.join(RELATIVE, 'checkpoint0018.hdf5'))

model.save(MODEL_DIR)

In [ ]:
img = ((transform.augment_images([cv.cvtColor(cv.imread(os.path.join(RELATIVE, '000001.jpg')), cv.COLOR_BGR2RGB)])[0]/255)*2)-1
list_img = []
list_img.append(img)
list_list_img = []
list_list_img.append(list_img)
model.predict(list_list_img)

In [ ]:
import matplotlib.pyplot as plt
plt.xticks([])
plt.yticks([])
plt.grid(False)
plt.imshow(transform.augment_images([cv.cvtColor(cv.imread(os.path.join(RELATIVE, '000001.jpg')), cv.COLOR_BGR2RGB)])[0]/255)
plt.show()

In [ ]:
((transform.augment_images([cv.cvtColor(cv.imread(os.path.join(RELATIVE, '000001.jpg')), cv.COLOR_BGR2RGB)])[0]/255)*2)-1

In [ ]:
str(1)+'.jpg'

In [ ]:
from segmentation_models import Unet

from os.path import join
import tensorflow.keras.optimizers as tfko
import imgaug.augmenters as iaa
import tensorflow.keras.callbacks as tfkc

from utils.data_load_segmentation import load
from utils.data_augmentation_segmentation import DataGenerator
from utils.data_transform import transform

from os.path import exists
from os import mkdir
from tensorflow.keras.utils import to_categorical
from imgaug import ALL
from numpy import sqrt


#RELATIVE = join('D:', 'EfficientNetB6')
RELATIVE = join('C:\\', 'Users', 'mauro', 'Documents',
                'Minsait', 'NoOneDrive', 'UNet')
#RELATIVE = join('F:/', 'Mauro', 'EfficientNetB6')


N = 30

TEST_PERCENTAGE = 1/6

train_images, train_labels, test_images, test_labels = load(
    RELATIVE, N, TEST_PERCENTAGE)

IMAGE_DIMENSIONS = (256, 256, 3)


EPOCHS = 200

BATCH_SIZE = 1

LEARNING_RATE = 0.03

LOSS = 'categorical_crossentropy'
OPTIMIZER = tfko.Adam(learning_rate=LEARNING_RATE)


def augmenters():

    def sometimes(aug): return iaa.Sometimes(0.5, aug)

    return iaa.Sequential(
        [
            iaa.Fliplr(0.5),
            iaa.Flipud(0.1),
            sometimes(
                iaa.Affine(scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
                           translate_percent={
                               "x": (-0.1, 0.1), "y": (-0.1, 0.1)},
                           order=[0, 1],
                           cval=(0, 255),
                           mode=ALL)),

        ],
        random_order=True)


CHECKPOINTS_DIR = join(RELATIVE, 'checkpoints')
if not exists(CHECKPOINTS_DIR):
    mkdir(CHECKPOINTS_DIR)


def lr_schedule(epoch):
    learning_rate = 0.001
    if epoch > 180:
        learning_rate *= 0.0005
    elif epoch > 160:
        learning_rate *= 0.001
    elif epoch > 120:
        learning_rate *= 0.01
    elif epoch > 80:
        learning_rate *= 0.1
    return learning_rate


callbacks = [
    tfkc.ModelCheckpoint(filepath=join(CHECKPOINTS_DIR, 'checkpoint{epoch:04d}.hdf5'),
                         monitor='val_loss',
                         save_weights_only=True,
                         save_best_only=True,
                         verbose=1),
    tfkc.LearningRateScheduler(lr_schedule),
    tfkc.ReduceLROnPlateau(factor=0.3,
                           patience=5,
                           min_lr=0.0000005,
                           verbose=1)
]


train_data = DataGenerator(train_images,
                           train_labels,
                           batch_size=BATCH_SIZE,
                           transform=transform(IMAGE_DIMENSIONS[:-1]),
                           image_dimensions=IMAGE_DIMENSIONS,
                           augmenters=augmenters(),
                           shuffle=True)

test_data = DataGenerator(test_images,
                          test_labels,
                          batch_size=BATCH_SIZE,
                            transform=transform(IMAGE_DIMENSIONS[:-1]),
                          image_dimensions=IMAGE_DIMENSIONS)


model = Unet(
        backbone_name='vgg16',
        input_shape=IMAGE_DIMENSIONS,
        classes=1,
        activation='sigmoid',
        weights=None,
        encoder_weights=None, #'imagenet',
        encoder_freeze=False,
        encoder_features='default',
        decoder_block_type='upsampling',
        decoder_filters=(256, 128, 64, 32, 16),
        decoder_use_batchnorm=True
)

model.summary()

model.compile(loss=LOSS,
              optimizer=OPTIMIZER,
              metrics=['accuracy'])

MODEL_DIR = join(RELATIVE, 'model.h5')

history = model.fit_generator(generator=train_data,
                              validation_data=test_data,
                              epochs=EPOCHS,
                              callbacks=callbacks)

model.save(MODEL_DIR)

scores = model.evaluate(test_data,
                        verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(train_data[0][0][0])

In [ ]:
plt.imshow(train_data[0][1][0])

In [ ]:
from segmentation_models import Unet

from os.path import join
import tensorflow.keras.optimizers as tfko
import imgaug.augmenters as iaa
import tensorflow.keras.callbacks as tfkc

from utils.data_load_segmentation import load
from utils.data_augmentation_segmentation import DataGenerator
from utils.data_transform_segmentation import transform

from os.path import exists
from os import mkdir
from tensorflow.keras.utils import to_categorical
from imgaug import ALL
from numpy import sqrt


#RELATIVE = join('D:', 'UNet')
RELATIVE = join('C:\\', 'Users', 'mauro', 'Documents',
                'Minsait', 'NoOneDrive', 'UNet2')
#RELATIVE = join('F:/', 'Mauro', 'UNet')


N = 642

TEST_PERCENTAGE = 1/6

train_images, train_labels, test_images, test_labels = load(
    RELATIVE, N, TEST_PERCENTAGE)

IMAGE_DIMENSIONS = (512, 512, 3)


EPOCHS = 200

BATCH_SIZE = 1

LEARNING_RATE = 0.0001

LOSS = 'binary_crossentropy'
OPTIMIZER = tfko.Adam(learning_rate=LEARNING_RATE)


def augmenters():

    def sometimes(aug): return iaa.Sometimes(0.99, aug)

    return iaa.Sequential(
        [
            iaa.Fliplr(0.5),
            iaa.Flipud(0.1),
            sometimes(
                iaa.Affine(scale={"x": (0.875, 1.125), "y": (0.875, 1.125)},
                           translate_percent={
                               "x": (-0.125, 0.125), "y": (-0.125, 0.125)},
                           order=[0, 1],
                           cval=0,
                           mode='constant')
            ),
            sometimes(iaa.Multiply((0.875, 1.125), per_channel=0.25))
        ],
        random_order=True)


CHECKPOINTS_DIR = join(RELATIVE, 'checkpoints')
if not exists(CHECKPOINTS_DIR):
    mkdir(CHECKPOINTS_DIR)


def lr_schedule(epoch):
    learning_rate = 0.001
    if epoch > 180:
        learning_rate *= 0.0005
    elif epoch > 160:
        learning_rate *= 0.001
    elif epoch > 120:
        learning_rate *= 0.01
    elif epoch > 80:
        learning_rate *= 0.1
    return learning_rate


callbacks = [
    tfkc.ModelCheckpoint(filepath=join(CHECKPOINTS_DIR, 'checkpoint{epoch:04d}.hdf5'),
                         monitor='val_loss',
                         save_weights_only=True,
                         save_best_only=True,
                         verbose=1),
    tfkc.ReduceLROnPlateau(factor=0.3,
                           patience=5,
                           min_lr=0.00000001,
                           verbose=1)
]


train_data = DataGenerator(train_images,
                           train_labels,
                           batch_size=BATCH_SIZE,
                           transform=transform(IMAGE_DIMENSIONS[:-1]),
                           image_dimensions=IMAGE_DIMENSIONS,
                           augmenters=augmenters(),
                           shuffle=True)

test_data = DataGenerator(test_images,
                          test_labels,
                          batch_size=BATCH_SIZE,
                          transform=transform(IMAGE_DIMENSIONS[:-1]),
                          image_dimensions=IMAGE_DIMENSIONS)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(25,25))
for i in range(16):
    xtrain_data = train_data[i]
    plt.subplot(8,4, 1+i*2)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(xtrain_data[0][0])
    plt.subplot(8,4, i*2+2)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(xtrain_data[1][0])
plt.show()

In [ ]:
from numpy.random import shuffle, randint
randint(0, 2147483648)
from imgaug import seed
seed(2147483649)

In [ ]:
model = Unet(
    backbone_name='vgg16',
    input_shape=IMAGE_DIMENSIONS,
    classes=1,
    activation='sigmoid',
    weights=None,
    encoder_weights=None,  # 'imagenet',
    encoder_freeze=False,
    encoder_features='default',
    decoder_block_type='upsampling',
    decoder_filters=(256, 128, 64, 32, 16),
    decoder_use_batchnorm=True
)
model.summary()

model.compile(loss=LOSS,
              optimizer=OPTIMIZER,
              metrics=['accuracy'])

In [ ]:
MODEL_DIR = join(RELATIVE, 'model.h5')

history = model.fit_generator(generator=train_data,
                              validation_data=test_data,
                              epochs=EPOCHS,
                              callbacks=callbacks)

model.save(MODEL_DIR)

scores = model.evaluate(test_data,
                        verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
from tensorflow.python.keras.utils.data_utils import Sequence

from numpy import floor, arange, array
from numpy.random import shuffle, randint
from cv2 import imread, cvtColor, COLOR_BGR2RGB, IMREAD_UNCHANGED
from imgaug.augmentables.segmaps import SegmentationMapsOnImage


class DataGenerator(Sequence):

    def __init__(self,
                 images_paths,
                 labels,
                 transform=None,
                 batch_size=64,
                 image_dimensions=(64, 64, 3),
                 shuffle=False,
                 augmenters=None):
        self.labels = labels
        self.images_paths = images_paths
        self.image_dimensions = image_dimensions
        self.transform = transform
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augmenters = augmenters
        self.on_epoch_end()

    def __len__(self):
        return int(floor(len(self.images_paths) / self.batch_size))

    def on_epoch_end(self):
        self.indexes = arange(len(self.images_paths))
        if self.shuffle:
            shuffle(self.indexes)

    def __getitem__(self,
                    index):
        indexes = self.indexes[index *
                               self.batch_size: (index + 1) * self.batch_size]
        images = [cvtColor(imread(str(self.images_paths[k][0])),
                           COLOR_BGR2RGB) for k in indexes]
        images_labels = [imread(str(self.labels[k][0]),
                                IMREAD_UNCHANGED) for k in indexes]
        if self.transform != None:
            images = self.transform.augment_images(images)
            images_labels = self.transform.augment_images(images_labels)
        if self.augmenters != None:
            self.augmenters.seed_(randint(0, 2147483648))
            images, images_labels_seg = self.augmenters(images=images, segmentation_maps=[SegmentationMapsOnImage(
                images_labels[k], shape=images_labels[k].shape) for k in range(len(indexes))])
            images_labels = [images_labels_seg[k].get_arr()
                             for k in range(len(images_labels_seg))]
        images = array([img/255.0 for img in images])
        images_labels = array([img/255 for img in images_labels])
        images = images.astype('float32')
        images_labels = images_labels.astype('float32')
        print(images)
        print(images_labels)
        return images, images_labels

In [ ]:
from segmentation_models import Unet

from os.path import join
import tensorflow.keras.optimizers as tfko
import imgaug.augmenters as iaa
import tensorflow.keras.callbacks as tfkc

from utils.data_load_segmentation import load
from utils.data_transform_segmentation import transform
from utils.losses import bce_dice_loss
from utils.metrics import iou_tf

from os.path import exists
from os import mkdir
from tensorflow.keras.utils import to_categorical


#RELATIVE = join('D:', 'EfficientNetB6')
RELATIVE = join('C:\\', 'Users', 'mauro', 'Documents',
                'Minsait', 'NoOneDrive', 'UNet')
#RELATIVE = join('F:/', 'Mauro', 'UNet')


N = 1410

TEST_PERCENTAGE = 1/6

train_images, train_labels, test_images, test_labels = load(
    RELATIVE, N, TEST_PERCENTAGE)

IMAGE_DIMENSIONS = (512, 512, 3)


EPOCHS = 300

BATCH_SIZE = 1

LEARNING_RATE = 0.001

LOSS = bce_dice_loss
OPTIMIZER = tfko.Adam(learning_rate=LEARNING_RATE)


def augmenters():

    def sometimes(aug): return iaa.Sometimes(0.5, aug)

    return iaa.Sequential(
        [
            iaa.Fliplr(0.5),
            iaa.Flipud(0.125),
            sometimes(
                iaa.Affine(scale={"x": (0.875, 1.125), "y": (0.875, 1.125)},
                           translate_percent={
                               "x": (-0.125, 0.125), "y": (-0.125, 0.125)},
                           order=[0, 1],
                           cval=0,
                           mode='constant')
            ),
            sometimes(iaa.Multiply((0.75, 1.25), per_channel=0.25))
        ],
        random_order=True)


CHECKPOINTS_DIR = join(RELATIVE, 'checkpoints')
if not exists(CHECKPOINTS_DIR):
    mkdir(CHECKPOINTS_DIR)


def lr_schedule(epoch):
    learning_rate = 0.001
    if epoch > 180:
        learning_rate *= 0.0005
    elif epoch > 160:
        learning_rate *= 0.001
    elif epoch > 120:
        learning_rate *= 0.01
    elif epoch > 80:
        learning_rate *= 0.1
    return learning_rate


callbacks = [
    tfkc.ModelCheckpoint(filepath=join(CHECKPOINTS_DIR, 'checkpoint{epoch:04d}.hdf5'),
                         monitor='val_loss',
                         save_weights_only=True,
                         save_best_only=True,
                         verbose=1),
    tfkc.ReduceLROnPlateau(factor=0.3,
                           patience=5,
                           min_lr=0.00000001,
                           verbose=1)
]


train_data = DataGenerator(train_images,
                           train_labels,
                           batch_size=BATCH_SIZE,
                           transform=transform(IMAGE_DIMENSIONS[:-1]),
                           image_dimensions=IMAGE_DIMENSIONS,
                           augmenters=augmenters(),
                           shuffle=True)

test_data = DataGenerator(test_images,
                          test_labels,
                          batch_size=BATCH_SIZE,
                          transform=transform(IMAGE_DIMENSIONS[:-1]),
                          image_dimensions=IMAGE_DIMENSIONS)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(25,25))
for i in range(16):
    xtrain_data = train_data[i]
    plt.subplot(8,4, 1+i*2)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(xtrain_data[0][0])
    plt.subplot(8,4, i*2+2)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(xtrain_data[1][0])
plt.show()

In [ ]:
model = Unet(
    backbone_name='efficientnetb5',
    input_shape=IMAGE_DIMENSIONS,
    classes=1,
    activation='sigmoid',
    weights=None,
    encoder_weights='imagenet',
    encoder_freeze=False,
    encoder_features='default',
    decoder_block_type='transpose',
    decoder_filters=(256, 128, 64, 32, 16),
    decoder_use_batchnorm=True
)
model.summary()

model.compile(loss=LOSS,
              optimizer=OPTIMIZER,
              metrics=[iou_tf])

MODEL_DIR = join(RELATIVE, 'model.h5')

history = model.fit_generator(generator=train_data,
                              validation_data=test_data,
                              epochs=EPOCHS,
                              callbacks=callbacks)

model.save(MODEL_DIR)

scores = model.evaluate(test_data,
                        verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


In [ ]:
from tensorflow.keras.models import load_model
from utils.losses import bce_dice_loss
import efficientnet.tfkeras

MODEL = load_model('UNet.h5', compile=False)

In [ ]:
MODEL.input.shape

In [ ]:
MODEL.input.shape[1]

In [ ]:
from segmentation_models import Unet

from os.path import join
from datetime import datetime

import tensorflow.keras.optimizers as tfko
import imgaug.augmenters as iaa
import tensorflow.keras.callbacks as tfkc

from Utils.ImageSegmentation import load, DataGenerator, resizeImage
from Utils.Losses import bce_dice_loss
from Utils.Metrics import iou

from os.path import exists
from os import mkdir


#RELATIVE = join('D:', 'EfficientNetB6')
RELATIVE = join('C:\\', 'Users', 'mauro', 'Documents',
                'Minsait', 'NoOneDrive', 'UNet')
#RELATIVE = join('F:/', 'Mauro', 'UNet')


N = 1410

TEST_PERCENTAGE = 1/6

CLASS_NAMES = ['smoke', 'fire']

train_images, train_labels, test_images, test_labels = load(
    RELATIVE, CLASS_NAMES, n=N, testPercentage=TEST_PERCENTAGE)

IMAGE_DIMENSIONS = (512, 512, 3)


EPOCHS = 256

BATCH_SIZE = 1

LEARNING_RATE = 0.001

LOSS = bce_dice_loss
OPTIMIZER = tfko.Adam(learning_rate=LEARNING_RATE)


def augmenters():

    def sometimes(aug): return iaa.Sometimes(0.675, aug)

    return iaa.Sequential(
        [
            iaa.Fliplr(0.5),
            iaa.Flipud(0.25),
            sometimes(
                iaa.Affine(scale={"x": (0.75, 1.25), "y": (0.75, 1.25)},
                           cval=0,
                           mode='constant')
            ),
            sometimes(
                iaa.Affine(translate_percent={
                    "x": (-0.25, 0.25), "y": (-0.25, 0.25)},
                    mode='edge')
            ),
            sometimes(iaa.Multiply((0.75, 1.25), per_channel=0.25)),
            sometimes(iaa.Pepper(p=(0.0, 0.125)))
        ],
        random_order=True)


CHECKPOINTS_DIR = join(RELATIVE, 'Checkpoints')
if not exists(CHECKPOINTS_DIR):
    mkdir(CHECKPOINTS_DIR)

#!rm -rf ./Logs/
log = join(RELATIVE, 'Logs')
if not exists(log):
    mkdir(log)
fit = join(log, 'Fit')
if not exists(fit):
    mkdir(fit)
LOG_DIR = join(fit, datetime.now().strftime('%Y%m%d-%H%M%S'))
if not exists(LOG_DIR):
    mkdir(LOG_DIR)


'''
def lr_schedule(epoch):
    learning_rate = 0.001
    if epoch > 180:
        learning_rate *= 0.0005
    elif epoch > 160:
        learning_rate *= 0.001
    elif epoch > 120:
        learning_rate *= 0.01
    elif epoch > 80:
        learning_rate *= 0.1
    return learning_rate
'''


callbacks = [
    tfkc.ModelCheckpoint(filepath=join(CHECKPOINTS_DIR, 'CheckpointEpoch{epoch:04d}ValLoss.hdf5'),
                         monitor='val_loss',
                         mode='min',
                         save_weights_only=True,
                         save_best_only=True,
                         verbose=1),
    tfkc.ReduceLROnPlateau(monitor='val_loss',
                           mode='min',
                           factor=0.5,
                           patience=4,
                           cooldown=4,
                           min_lr=0.0,
                           min_delta=0.0,
                           verbose=1),
    tfkc.ReduceLROnPlateau(monitor='val_loss',
                           mode='min',
                           factor=0.5,
                           patience=16,
                           cooldown=0,
                           min_lr=0.0,
                           min_delta=0.0,
                           verbose=1),
    # tfkc.LearningRateScheduler(lr_schedule,
    #                           verbose=1),
    tfkc.TensorBoard(log_dir=LOG_DIR,
                     histogram_freq=1,
                     profile_batch=0,
                     write_graph=True,
                     write_images=True,
                     update_freq='epoch',
                     embeddings_freq=0,
                     embeddings_metadata=None)
]


train_data = DataGenerator(train_images,
                           train_labels,
                           batchSize=BATCH_SIZE,
                           resize=resizeImage(IMAGE_DIMENSIONS[:-1]),
                           imageDimensions=IMAGE_DIMENSIONS,
                           augmenters=augmenters(),
                           shuffle=True)

test_data = DataGenerator(test_images,
                          test_labels,
                          batchSize=BATCH_SIZE,
                          resize=resizeImage(IMAGE_DIMENSIONS[:-1]),
                          imageDimensions=IMAGE_DIMENSIONS)

In [ ]:
train_labels

In [ ]:
train_images

In [ ]:
train_data[0][0].shape

In [ ]:
import cv2 as cv
from numpy import frombuffer, uint8, float32, array, where, amax, zeros
img1=cv.imread('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\UNet\\Data\\Train\\Images\\smoke_(249).png', cv.IMREAD_UNCHANGED)
img2=cv.imread('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\UNet\\Data\\Train\\Labels\\fire\\smoke_(249).png', 7)
imgs=[img1,img2]

img2.shape

In [ ]:
import matplotlib.pyplot as plt
from numpy import frombuffer, uint8, float32, array, where, amax, zeros

plt.figure(figsize=(25,25))
for i in range(16):
    xtrain_data = train_data[i]
    plt.subplot(8,4, i*2+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(xtrain_data[0][0])
    plt.subplot(8,4, i*2+2)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(xtrain_data[1][0])
    print(xtrain_data[1][0][0][where(
        xtrain_data[1][0][0] == amax(xtrain_data[1][0][0]))][0])
plt.show()

In [ ]:



model = Unet(backbone_name='efficientnetb5',
             input_shape=IMAGE_DIMENSIONS,
             classes=2,
             activation='sigmoid',
             weights=None,
             encoder_weights=None,
             encoder_freeze=False,
             encoder_features='default',
             decoder_block_type='transpose',
             decoder_filters=(256, 128, 64, 32, 16),
             decoder_use_batchnorm=True)
model.summary()

model.compile(loss=LOSS,
              optimizer=OPTIMIZER,
              metrics=[iou])


MODEL_DIR = join(RELATIVE, 'model.h5')

In [ ]:
print(model.metrics_names)

In [ ]:
layers = model.layers

for layer in layers:
    name = layer.name
    input_shape = layer.input_shape
    output_shape = layer.output_shape
    print('%s   input shape: %s, output_shape: %s.\n' % (name, input_shape, output_shape))

In [ ]:


history = model.fit(train_data,
                    epochs=1,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=test_data,
                    shuffle=True,
                    workers=1,
                    use_multiprocessing=False)

model.save(MODEL_DIR)



In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.backend import clear_session
# from tensorflow.keras.models import model_from_json
import efficientnet.tfkeras

from datetime import datetime
from numpy import frombuffer, uint8, float32, array, where, amax, zeros
from cv2 import imdecode, imwrite, imencode, threshold, THRESH_BINARY, merge, bitwise_not, bitwise_and, bitwise_or, addWeighted, imread
from base64 import b64decode, b64encode
from os.path import join, exists
from os import mkdir, environ
from random import randint

image = imread('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\UNet\\Data\\Train\\Images\\smoke_(249).jpg')
(x, y, _) = image.shape

imageTransformed = resizeImage([512, 512]).augment_images([image])

imagePrepared = array(
    [img/255.0 for img in imageTransformed]).astype(float32)

imagePredicted = [imagePrepared]

before = datetime.now()
prediction = model.predict(imagePredicted)
after = datetime.now()
timePrediction = after - before



In [ ]:
prediction[0][..., 1].shape

In [ ]:
listPrediction = []
for i in range(prediction.shape[-1]):
    imagePrediction = (prediction[0][..., i]*255.0).astype(uint8)

    _, thresh = threshold(imagePrediction, 127, 255, THRESH_BINARY)
    thresh = thresh.astype(uint8)

    mask = bitwise_not(thresh)

    blankImage = zeros((512, 512, 3), uint8)
    blankImage[:, 0:3] = (
        randint(0, 255), randint(0, 255), randint(0, 255))

    colortrans = addWeighted(imageTransformed[0], 0.7, bitwise_and(bitwise_or(merge([mask, mask, mask]), bitwise_and(
        merge([thresh, thresh, thresh]), blankImage, None), None), imageTransformed[0], None), 0.4, 1)

    imageRequest = None
    if x <= y:
        w = x/y
        thresh = thresh[int((512-512*w)/2):int(-((512-512*w)/2)-1), :]
        imageRequest = colortrans[int(
            (512-512*w)/2):int(-((512-512*w)/2)-1), :]
    else:
        w = y/x
        thresh = thresh[:, int((512-512*w)/2):int(-((512-512*w)/2)-1)]
        imageRequest = colortrans[:, int((512-512*w)/2):int(-((512-512*w)/2)-1)]

    _, data = imencode('.jpg', imageRequest)

    listPrediction.append(str(i))
    listPrediction.append((b64encode(data.tobytes())).decode('ascii'))

    imwrite('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\UNet\\Data\\Train\\smoke_(249).jpg', image)
    imwrite('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\UNet\\Data\\Train\\smoke_(249)mask.png', thresh)
    imwrite('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\UNet\\Data\\Train\\smoke_(249)new.jpg', imageRequest)

In [ ]:
listPrediction

In [ ]:
from segmentation_models import Unet

from os.path import join
from datetime import datetime

import tensorflow.keras.optimizers as tfko
import imgaug.augmenters as iaa
import tensorflow.keras.callbacks as tfkc

from Utils.ImageSegmentation import load, DataGenerator, resizeImage
from Utils.Losses import bce_dice_loss
from Utils.Metrics import iou

from os.path import exists
from os import mkdir


#Relative = join('D:\\', 'UNet', 'EfficientNetB6')
Relative = join('C:\\', 'Users', 'mauro', 'Documents',
                'Minsait', 'NoOneDrive', 'UNet', 'EfficientNetB7')
#Relative = join('F:\\', 'Mauro', 'UNet', 'EfficientNetB6')


n = 1410

testPercentage = 1/6

classNames = ['smoke']


trainImages, trainLabels, testImages, testLabels = load(
    Relative, classNames)


imageDimensions = (512, 512, 3)

batchSize = 1

learningRate = 0.001

optimizer = tfko.Adam(learning_rate=learningRate)

loss = bce_dice_loss

epochs = 256


def augmenters():

    def sometimes(aug): return iaa.Sometimes(0.675, aug)

    return iaa.Sequential(
        [
            iaa.Fliplr(0.5),
            iaa.Flipud(0.25),
            sometimes(
                iaa.Affine(scale={"x": (0.75, 1.25), "y": (0.75, 1.25)},
                           cval=0,
                           mode='constant')
            ),
            sometimes(
                iaa.Affine(translate_percent={
                    "x": (-0.25, 0.25), "y": (-0.25, 0.25)},
                    mode='edge')
            ),
            sometimes(iaa.Multiply((0.75, 1.25), per_channel=0.25)),
            sometimes(iaa.Pepper(p=(0.0, 0.125)))
        ],
        random_order=True)


checkpointsDir = join(Relative, 'Checkpoints')
if not exists(checkpointsDir):
    mkdir(checkpointsDir)

#!rm -rf ./Logs/
logsDir = join(Relative, 'Logs')
if not exists(logsDir):
    mkdir(logsDir)
fitDir = join(logsDir, 'Fit')
if not exists(fitDir):
    mkdir(fitDir)
logDir = join(fitDir, datetime.now().strftime('%Y%m%d-%H%M%S'))
if not exists(logDir):
    mkdir(logDir)

'''
def lrSchedule(epoch):
    learningRate = 0.001
    if epoch > 180:
        learningRate *= 0.0005
    elif epoch > 160:
        learningRate *= 0.001
    elif epoch > 120:
        learningRate *= 0.01
    elif epoch > 80:
        learningRate *= 0.1
    return learningRate
'''

callbacks = [
    tfkc.ModelCheckpoint(filepath=join(checkpointsDir, 'CheckpointEpoch{epoch:04d}Accuracy{val_iou:.4f}.hdf5'),
                         monitor='val_loss',
                         mode='min',
                         save_weights_only=True,
                         save_best_only=True,
                         verbose=1),
    tfkc.ReduceLROnPlateau(monitor='val_loss',
                           mode='min',
                           factor=0.5,
                           patience=4,
                           cooldown=4,
                           min_lr=0.0,
                           min_delta=0.0,
                           verbose=1),
    tfkc.ReduceLROnPlateau(monitor='val_loss',
                           mode='min',
                           factor=0.5,
                           patience=16,
                           cooldown=0,
                           min_lr=0.0,
                           min_delta=0.0,
                           verbose=1),
    # tfkc.LearningRateScheduler(lrSchedule,
    #                           verbose=1),
    tfkc.TensorBoard(log_dir=logDir,
                     histogram_freq=1,
                     profile_batch=0,
                     write_graph=True,
                     write_images=True,
                     update_freq='epoch',
                     embeddings_freq=0,
                     embeddings_metadata=None)
]


trainData = DataGenerator(trainImages,
                          trainLabels,
                          batchSize=batchSize,
                          resize=resizeImage(imageDimensions[:-1]),
                          imageDimensions=imageDimensions,
                          augmenters=augmenters(),
                          shuffle=True)

testData = DataGenerator(testImages,
                         testLabels,
                         batchSize=batchSize,
                         resize=resizeImage(imageDimensions[:-1]),
                         imageDimensions=imageDimensions)




In [ ]:
len(testLabels[1])

In [ ]:
from numpy import array
array(range(1, 5))

In [ ]:
testImages.shape

In [ ]:
model = Unet(backbone_name='efficientnetb7',
             input_shape=imageDimensions,
             classes=len(classNames),
             activation='sigmoid',
             weights=None,
             encoder_weights='imagenet',
             encoder_freeze=False,
             encoder_features='default',
             decoder_block_type='transpose',
             decoder_filters=(256, 128, 64, 32, 16),
             decoder_use_batchnorm=True)
model.summary()


model.compile(loss=loss,
              optimizer=optimizer,
              metrics=[iou])

history = model.fit(trainData,
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=testData,
                    shuffle=True,
                    workers=1,
                    use_multiprocessing=False)


In [ ]:
from glob import glob
from os.path import join

from tensorflow.python.keras.utils.data_utils import Sequence
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

from imgaug.augmenters import Sequential
from imgaug.augmenters.size import CenterPadToAspectRatio, Resize

from numpy import hsplit, asarray, array, floor, arange, zeros, uint8
from numpy.random import shuffle
from cv2 import imread, cvtColor, COLOR_BGR2RGB, IMREAD_UNCHANGED, INTER_AREA

import matplotlib.pyplot as plt

from numpy import argmax, around, newaxis
from sklearn.metrics import confusion_matrix
from itertools import product


def load(relativePath, labels, n=256, validationPercentage=0.5):

    filenamesImagesTrain = list(
        glob(join(relativePath, 'Data', 'Train', 'Images', '*')))
    filenamesListLabelsTrain = []
    for l in labels:
        filenamesListLabelsTrain.append(
            list(glob(join(relativePath, 'Data', 'Train', 'Labels', l, '*'))))

    filenamesImagesValidation = list(
        glob(join(relativePath, 'Data', 'Validation', 'Images', '*')))
    filenamesListLabelsValidation = []
    for l in labels:
        filenamesListLabelsValidation.append(
            list(glob(join(relativePath, 'Data', 'Validation', 'Labels', l, '*'))))

    dataTrain = []
    for i in range(int(n*(1-validationPercentage))):
        data = []
        data.append(filenamesImagesTrain[i])
        for l in filenamesListLabelsTrain:
            data.append(l[i])
        dataTrain.append(data)
    shuffle(dataTrain)

    dataValidation = []
    for i in range(int(n*validationPercentage)):
        data = []
        data.append(filenamesImagesValidation[i])
        for l in filenamesListLabelsValidation:
            data.append(l[i])
        dataValidation.append(data)
    shuffle(dataValidation)

    def splitLabelsImages(data, labels):
        dataSplit = hsplit(asarray(data), array(range(1, len(labels)+1)))
        images = dataSplit[0]
        labels = dataSplit[1:]
        return images, labels

    trainImages, trainLabels = splitLabelsImages(dataTrain, labels)

    validationImages, validationLabels = splitLabelsImages(
        dataValidation, labels)

    return trainImages, trainLabels, validationImages, validationLabels


def prepareLabels(indexes, imagesLabels):
    imagesLabelsList = []
    for k in range(len(indexes)):
        img = None
        for i in range(len(imagesLabels)):
            imgLabel = imagesLabels[i][k]
            if img is None:
                img = zeros(
                    (imgLabel.shape[0], imgLabel.shape[1], len(imagesLabels)), uint8)
            for x in range(len(imgLabel)):
                for y in range(len(imgLabel[x])):
                    img[x][y][i] = imgLabel[x][y]
        imagesLabelsList.append(img/255.0)
    return array(imagesLabelsList).astype('float32')


class DataGenerator(Sequence):

    def __init__(self,
                 imagesPaths,
                 labels,
                 resize=None,
                 batchSize=64,
                 imageDimensions=(64, 64, 3),
                 shuffle=False,
                 augmenters=None):
        self.labels = labels
        self.imagesPaths = imagesPaths
        self.imageDimensions = imageDimensions
        self.resize = resize
        self.batchSize = batchSize
        self.shuffle = shuffle
        self.augmenters = augmenters
        self.on_epoch_end()

    def __len__(self):
        return int(floor(len(self.imagesPaths) / self.batchSize))

    def on_epoch_end(self):
        self.indexes = arange(len(self.imagesPaths))
        if self.shuffle:
            shuffle(self.indexes)

    def __getitem__(self,
                    index):
        indexes = self.indexes[index *
                               self.batchSize: (index + 1) * self.batchSize]
        images = [cvtColor(imread(str(self.imagesPaths[k][0])),
                           COLOR_BGR2RGB) for k in indexes]
        imagesLabels = [[imread(str(l[k][0]), IMREAD_UNCHANGED)
                         for k in indexes] for l in self.labels]
        if self.resize != None:
            images = self.resize.augment_images(images)
            imagesLabels = [self.resize.augment_segmentation_maps([SegmentationMapsOnImage(
                l[k], shape=l[k].shape) for k in range(len(indexes))]) for l in imagesLabels]
        if self.augmenters != None:
            self.augmenters = self.augmenters.to_deterministic()
            images = self.augmenters.augment_images(images)
            imagesLabelsSeg = [self.augmenters.augment_segmentation_maps([SegmentationMapsOnImage(
                l[k], shape=l[k].shape) for k in range(len(indexes))]) for l in imagesLabels]
            imagesLabels = [[lsi.get_arr() for lsi in ls]
                            for ls in imagesLabelsSeg]
        images = array([img/255.0 for img in images])
        imagesLabels = prepareLabels(indexes, imagesLabels)
        images = images.astype('float32')
        return images, imagesLabels


def resizeImage(imageDimensions):
    return Sequential(
        [
            CenterPadToAspectRatio(float(imageDimensions[0])/float(imageDimensions[1]),
                                   pad_mode='constant',
                                   pad_cval=0),
            Resize(imageDimensions,
                   interpolation=INTER_AREA)
        ])


def imageGrid(images, labels, classNames=['0'], perRow=2, imageDimensions=(64, 64), nImages=16):
    nClasses = len(classNames)+1
    figure = plt.figure(figsize=imageDimensions)
    resize = resizeImage(imageDimensions)
    columns = perRow*nClasses
    rows = nImages/columns
    for i in range(nImages/nClasses):
        plt.subplot(columns, rows, (i*nClasses)+1,
                    title=(images[i][0].split('\\')[-1]).split('.')[0])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(resize.augment_image(
            cvtColor(imread(images[i][0]), COLOR_BGR2RGB)))
        for j in range(nClasses-1):
            plt.subplot(columns, rows, (i*nClasses)+1+j, title=classNames[j])
            plt.xticks([])
            plt.yticks([])
            plt.grid(False)
            mask = imread(labels[j][i][0], IMREAD_UNCHANGED)
            plt.imshow(resize.augment_segmentation_maps(
                [SegmentationMapsOnImage(mask, shape=mask.shape)])[0])
    return figure


def confusionMatrix(model, validationImages, validationLabels, classNames, imageDimensions, epoch):
    resize = resizeImage(imageDimensions)
    imagesLabels = [[imread(str(l[0]), IMREAD_UNCHANGED)
                     for l in ls] for ls in validationLabels]
    imagesLabels = [resize.augment_segmentation_maps(
        [SegmentationMapsOnImage(l, shape=l.shape) for l in ls]) for ls in imagesLabels]
    imagesLabels = prepareLabels(range(validationLabels), imagesLabels)
    cm = confusion_matrix(imagesLabels, argmax(model.predict([resize.augment_images([cvtColor(
        imread(image), COLOR_BGR2RGB).astype('float32')/255.0 for image in validationImages])]), axis=1))
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest')
    plt.title('Confusion matrix')
    plt.colorbar()
    tickMarks = arange(len(classNames))
    plt.xticks(tickMarks, classNames, rotation=45)
    plt.yticks(tickMarks, classNames)
    cm = around(cm.astype('float') / cm.sum(axis=1)[:, newaxis], decimals=2)
    for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment='center',
                 color='white' if cm[i, j] > cm.max() / 2.0 else 'black')
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure


In [ ]:
from segmentation_models import Unet

from segmentation_models.losses import categorical_focal_dice_loss, binary_focal_dice_loss
from segmentation_models.metrics import IOUScore, FScore

from os.path import join

from Utils.ImageSegmentation import load, DataGenerator, resizeImage

import tensorflow.keras.optimizers as tfko
import imgaug.augmenters as iaa
import tensorflow.keras.callbacks as tfkc

import Utils.Callbacks as ca
from tensorflow.summary import create_file_writer, image
from Utils.ImageSegmentation import imageGrid, confusionMatrix
from os.path import exists
from os import mkdir
from datetime import datetime


#Relative = join('D:\\', 'UNet', 'EfficientNetB7')
Relative = join('C:\\', 'Users', 'mauro', 'Documents',
                'Minsait', 'NoOneDrive', 'UNet', 'EfficientNetB7')
#Relative = join('F:\\', 'Mauro', 'UNet', 'EfficientNetB7')

nameLogs = 'UNetBkEffB7'


n = 128

validationPercentage = 1/6

classNames = ['smoke']


trainImages, trainLabels, validationImages, validationLabels = load(
    Relative, classNames, n=n, validationPercentage=validationPercentage)


imageDimensions = (512, 512, 3)

batchSize = 1

backbone = 'efficientnetb3'

learningRate = 0.001

optimizer = tfko.Adam(learning_rate=learningRate)

metrics = [IOUScore(threshold=0.5), FScore(threshold=0.5)]

epochs = 4


def augmenters():

    def sometimes(aug): return iaa.Sometimes(0.675, aug)

    return iaa.Sequential(
        [
            iaa.Fliplr(0.5),
            iaa.Flipud(0.25),
            sometimes(
                iaa.Affine(scale={"x": (0.75, 1.25), "y": (0.75, 1.25)},
                           cval=0,
                           mode='constant')
            ),
            sometimes(
                iaa.Affine(translate_percent={
                    "x": (-0.25, 0.25), "y": (-0.25, 0.25)},
                    mode='edge')
            ),
            sometimes(iaa.Multiply((0.75, 1.25), per_channel=0.25)),
            sometimes(iaa.Pepper(p=(0.0, 0.25)))
        ],
        random_order=True)


checkpointsDir = join(Relative, 'Checkpoints')
if not exists(checkpointsDir):
    mkdir(checkpointsDir)

logsDir = join(Relative, 'Logs')
if not exists(logsDir):
    mkdir(logsDir)
fitDir = join(logsDir, 'Fit')
if not exists(fitDir):
    mkdir(fitDir)
logDir = join(fitDir, nameLogs+datetime.now().strftime('%Y%m%d%H%M%S'))
if not exists(logDir):
    mkdir(logDir)

fileWriter = create_file_writer(logDir)
fileWriter.set_as_default()

with fileWriter.as_default():
    image("Training data",
          ca.plotImages(imageGrid(trainImages,
                                  trainLabels,
                                  classNames=classNames,
                                  perRow=1,
                                  imageDimensions=(32, 32),
                                  nImages=32)),
          step=0)

with fileWriter.as_default():
    image("Validation data",
          ca.plotImages(imageGrid(validationImages,
                                  validationLabels,
                                  classNames=classNames,
                                  perRow=1,
                                  imageDimensions=(32, 32),
                                  nImages=16)),
          step=0)


'''
def lrSchedule(epoch):
    learningRate = 0.001
    if epoch > 180:
        learningRate *= 0.0005
    elif epoch > 160:
        learningRate *= 0.001
    elif epoch > 120:
        learningRate *= 0.01
    elif epoch > 80:
        learningRate *= 0.1
    scalar('Learning Rate', data=learningRate, step=epoch)
    return learningRate
'''

callbacks = [
    tfkc.ModelCheckpoint(filepath=join(checkpointsDir, 'CheckpointEpoch{epoch:04d}Accuracy{val_iou_score:.4f}.hdf5'),
                         monitor='val_iou_score',
                         mode='max',
                         save_weights_only=True,
                         save_best_only=True,
                         verbose=1),
    ca.ReduceLR(monitor='val_iou_score',
                mode='max',
                factor=0.5,
                patience=1,
                cooldown=4,
                min_lr=0.0,
                min_delta=0.0,
                verbose=1),
    ca.ReduceLR(monitor='val_iou_score',
                mode='max',
                factor=0.5,
                patience=16,
                cooldown=0,
                min_lr=0.0,
                min_delta=0.0,
                verbose=1),
    # tfkc.LearningRateScheduler(lrSchedule,
    #                           verbose=1),
    tfkc.TensorBoard(log_dir=logDir,
                     histogram_freq=1,
                     profile_batch=2,
                     write_graph=True,
                     write_images=True,
                     update_freq='epoch',
                     embeddings_freq=0,
                     embeddings_metadata=None)
]
'''
write_grads,
embedding_layer_names,
embeddings_data
'''
trainData = DataGenerator(trainImages,
                          trainLabels,
                          batchSize=batchSize,
                          resize=resizeImage(imageDimensions[:-1]),
                          imageDimensions=imageDimensions,
                          augmenters=augmenters(),
                          shuffle=True)

validationData = DataGenerator(validationImages,
                               validationLabels,
                               batchSize=batchSize,
                               resize=resizeImage(imageDimensions[:-1]),
                               imageDimensions=imageDimensions)


model = Unet(backbone_name=backbone,
             input_shape=imageDimensions,
             classes=len(classNames),
             activation='sigmoid' if len(classNames) == 1 else 'softmax',
             encoder_weights=None,  # imagenet
             decoder_block_type='upsampling')  # transpose : -Conv2D
model.summary()


model.compile(loss=binary_focal_dice_loss if len(classNames) == 1 else categorical_focal_dice_loss,
              optimizer=optimizer,
              metrics=[metrics])


def logConfusionMatrix(epoch, logs):
    with fileWriter.as_default():
        image('Confusion Matrix', ca.plotImages(confusionMatrix(model, validationImages, validationLabels, classNames, imageDimensions[:2])), step=epoch)

        
callbacks.append(tfkc.LambdaCallback(on_epoch_end=logConfusionMatrix))


history = model.fit(trainData,
                    validation_data=validationData,
                    epochs=epochs,
                    callbacks=callbacks)

In [ ]:
validationLabels[:,1]

In [ ]:
%tensorboard --logdir ./logs/test

In [ ]:
print(join(fitDir, nameLogs+datetime.now().strftime('%Y%m%d%H%M%S')))

In [ ]:
from glob import glob
from os.path import join

from tensorflow.python.keras.utils.data_utils import Sequence
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

from imgaug.augmenters import Sequential
from imgaug.augmenters.size import CenterPadToAspectRatio, Resize

from numpy import hsplit, asarray, array, floor, arange, zeros, uint8
from numpy.random import shuffle
from cv2 import imread, cvtColor, COLOR_BGR2RGB, IMREAD_UNCHANGED, INTER_AREA

import matplotlib.pyplot as plt

from cv2 import threshold, THRESH_BINARY
from numpy import argmax, around, newaxis, concatenate
from sklearn.metrics import confusion_matrix
from itertools import product


def load(relativePath, labels, n=256, validationPercentage=0.5):

    filenamesImagesTrain = list(
        glob(join(relativePath, 'Data', 'Train', 'Images', '*')))
    filenamesListLabelsTrain = []
    for l in labels:
        filenamesListLabelsTrain.append(
            list(glob(join(relativePath, 'Data', 'Train', 'Labels', l, '*'))))

    filenamesImagesValidation = list(
        glob(join(relativePath, 'Data', 'Validation', 'Images', '*')))
    filenamesListLabelsValidation = []
    for l in labels:
        filenamesListLabelsValidation.append(
            list(glob(join(relativePath, 'Data', 'Validation', 'Labels', l, '*'))))

    dataTrain = []
    for i in range(int(n*(1-validationPercentage))):
        data = []
        data.append(filenamesImagesTrain[i])
        for l in filenamesListLabelsTrain:
            data.append(l[i])
        dataTrain.append(data)
    shuffle(dataTrain)

    dataValidation = []
    for i in range(int(n*validationPercentage)):
        data = []
        data.append(filenamesImagesValidation[i])
        for l in filenamesListLabelsValidation:
            data.append(l[i])
        dataValidation.append(data)
    shuffle(dataValidation)

    def splitLabelsImages(data, labels):
        dataSplit = hsplit(asarray(data), array(range(1, len(labels)+1)))
        images = dataSplit[0]
        labels = dataSplit[1:]
        return images, labels

    trainImages, trainLabels = splitLabelsImages(dataTrain, labels)

    validationImages, validationLabels = splitLabelsImages(
        dataValidation, labels)

    return trainImages, trainLabels, validationImages, validationLabels


def prepareLabels(indexes, imagesLabels):
    imagesLabelsList = []
    for k in range(len(indexes)):
        img = None
        for i in range(len(imagesLabels)):
            imgLabel = imagesLabels[i][k]
            if img is None:
                img = zeros(
                    (imgLabel.shape[0], imgLabel.shape[1], len(imagesLabels)), uint8)
            for x in range(len(imgLabel)):
                for y in range(len(imgLabel[x])):
                    img[x][y][i] = imgLabel[x][y]
        imagesLabelsList.append(img/255.0)
    return array(imagesLabelsList).astype('float32')


class DataGenerator(Sequence):

    def __init__(self,
                 imagePaths,
                 labelPaths,
                 resize=None,
                 batchSize=64,
                 imageDimensions=(64, 64, 3),
                 shuffle=False,
                 augmenters=None):
        self.labelPaths = labelPaths
        self.imagePaths = imagePaths
        self.imageDimensions = imageDimensions
        self.resize = resize
        self.batchSize = batchSize
        self.shuffle = shuffle
        self.augmenters = augmenters
        self.on_epoch_end()

    def __len__(self):
        return int(floor(len(self.imagePaths) / self.batchSize))

    def on_epoch_end(self):
        self.indexes = arange(len(self.imagePaths))
        if self.shuffle:
            shuffle(self.indexes)

    def __getitem__(self,
                    index):
        indexes = self.indexes[index *
                               self.batchSize: (index + 1) * self.batchSize]
        images = [cvtColor(imread(str(self.imagePaths[k][0])),
                           COLOR_BGR2RGB) for k in indexes]
        imagesLabels = [[imread(str(l[k][0]), IMREAD_UNCHANGED)
                         for k in indexes] for l in self.labelPaths]
        if self.resize != None:
            images = self.resize.augment_images(images)
            imagesLabels = [[lsi.get_arr() for lsi in ls] for ls in [self.resize.augment_segmentation_maps(
                [SegmentationMapsOnImage(l[k], shape=l[k].shape) for k in range(len(indexes))]) for l in imagesLabels]]
        if self.augmenters != None:
            self.augmenters = self.augmenters.to_deterministic()
            images = self.augmenters.augment_images(images)
            imagesLabels = [[lsi.get_arr() for lsi in ls] for ls in [self.augmenters.augment_segmentation_maps(
                [SegmentationMapsOnImage(l[k], shape=l[k].shape) for k in range(len(indexes))]) for l in imagesLabels]]
        images = array([img/255.0 for img in images]).astype('float32')
        imagesLabels = prepareLabels(indexes, imagesLabels)
        return images, imagesLabels


def resizeImage(imageDimensions):
    return Sequential(
        [
            CenterPadToAspectRatio(float(imageDimensions[0])/float(imageDimensions[1]),
                                   pad_mode='constant',
                                   pad_cval=0),
            Resize(imageDimensions,
                   interpolation=INTER_AREA)
        ])


def imageGrid(images, labels, classNames=['0'], perRow=2, imageDimensions=(64, 64), nImages=16):
    nClasses = len(classNames)+1
    resize = resizeImage(imageDimensions)
    columns = perRow*nClasses
    rows = int((nImages*nClasses)/columns)
    figure, axarr = plt.subplots(rows, columns, figsize=(
        imageDimensions[0], int(imageDimensions[1]*rows/columns)))
    k = 0
    for i in range(rows):
        for j in range(columns):
            if j % nClasses == 0:
                k = int(j/nClasses)+(i*perRow)
                axarr[i, j].imshow(resize.augment_image(
                    cvtColor(imread(images[k][0]), COLOR_BGR2RGB)))
                axarr[i, j].set_title(
                    (images[k][0].split('\\')[-1]).split('.')[0])
            else:
                mask = imread(labels[j % nClasses-1][k][0], IMREAD_UNCHANGED)
                axarr[i, j].imshow(resize.augment_segmentation_maps(
                    [SegmentationMapsOnImage(mask, shape=mask.shape)])[0].get_arr())
                axarr[i, j].set_title(classNames[j % nClasses-1])
            axarr[i, j].axis(False)
    plt.subplots_adjust(wspace=0, hspace=0.0675,
                        left=0, right=1, bottom=0, top=1)
    return figure


def confusionMatrix(model, validationImages, validationLabels, classNames, imageDimensions):
    resize = resizeImage(imageDimensions)
    images = [model.predict([resize.augment_images([cvtColor(imread(image[0]), COLOR_BGR2RGB).astype(
        'float32')/255.0])], batch_size=1) for image in validationImages]
    imagePredictions = []
    for i in range(images.shape[-1]):
        for p in [(img[0][..., i]*255.0).astype(uint8) for img in images]:
            _, thresh = threshold(p, 127, 255, THRESH_BINARY)
            imagePredictions.append(thresh/255)
    if len(classNames) == 1:
        cm = confusion_matrix((prepareLabels(range(len(validationImages)), [[lsi.get_arr() for lsi in ls] for ls in [resize.augment_segmentation_maps([SegmentationMapsOnImage(l, shape=l.shape) for l in ls]) for ls in [[imread(str(
            l[0]), IMREAD_UNCHANGED) for l in ls] for ls in validationLabels]]]).flatten()).astype(uint8), (array(imagePredictions).flatten()).astype(uint8))
    else:
        images = model.predict([resize.augment_images([cvtColor(imread(
            image[0]), COLOR_BGR2RGB).astype('float32')/255.0 for image in validationImages])])
        labels = prepareLabels(range(len(validationImages)), [[lsi.get_arr() for lsi in ls] for ls in [resize.augment_segmentation_maps(
            [SegmentationMapsOnImage(l, shape=l.shape) for l in ls]) for ls in [[imread(str(l[0]), IMREAD_UNCHANGED) for l in ls] for ls in validationLabels]]])
        cm = confusion_matrix(
            labels.reshape(-1, labels.shape[-1]), argmax(images.reshape(-1, images.shape[-1]), axis=1))
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest')
    plt.title('Confusion matrix')
    plt.colorbar()
    classNames = concatenate([['empty'], classNames])
    tickMarks = arange(len(classNames))
    plt.xticks(tickMarks, classNames, rotation=45)
    plt.yticks(tickMarks, classNames)
    cm = around(cm.astype('float') / cm.sum(axis=1)[:, newaxis], decimals=2)
    for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment='center',
                 color='white' if cm[i, j] > cm.max() / 2.0 else 'black')
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure


In [ ]:
model.input.shape

In [ ]:
resize = resizeImage(imageDimensions[:2])

In [ ]:
lista = array([resize.augment_image(cvtColor(imread(image[0]), COLOR_BGR2RGB)) for image in validationImages]).astype('float32')/255.0

In [ ]:
lista.shape

In [ ]:
predict = model.predict(lista, batch_size=1)

In [ ]:
resize = resizeImage(imageDimensions[:2])
labels = prepareLabels(range(len(validationImages)), [[lsi.get_arr() for lsi in ls] for ls in [resize.augment_segmentation_maps([SegmentationMapsOnImage(l, shape=l.shape) for l in ls]) for ls in [[imread(str(
            l[0]), IMREAD_UNCHANGED) for l in ls] for ls in validationLabels]]]).flatten().astype(uint8)

In [ ]:
images = model.predict([resize.augment_images([cvtColor(imread(image[0]), COLOR_BGR2RGB).astype('float32')/255.0 for image in validationImages])]).flatten()

In [ ]:
for i in range(prediction.shape[-1]):
    imagePredictions = [(prediction[0][..., i]*255.0).astype(uint8) for p in prediction]

    _, thresh = threshold(imagePrediction, 127, 255, THRESH_BINARY)

In [ ]:
labels

In [ ]:
import numpy
numpy.array([[21,312],[312,9]]).shape

In [ ]:
trainData[0][0].shape

In [ ]:

resize = resizeImage(imageDimensions[:2])
labels = prepareLabels(range(len(validationImages)), [[lsi.get_arr() for lsi in ls] for ls in [resize.augment_segmentation_maps([SegmentationMapsOnImage(l, shape=l.shape) for l in ls]) for ls in [[imread(str(
    l[0]), IMREAD_UNCHANGED) for l in ls] for ls in validationLabels]]])

In [ ]:
labels.shape

In [ ]:
arr = labels
print(arr.shape)

another_arr = arr.reshape(-1, arr.shape[-1])
print(another_arr.shape)

In [ ]:
arr.flatten().shape

In [ ]:
from numpy.random import rand
import matplotlib.pyplot as plt

test_data = [rand(10,10), rand(10,10),rand(10,10), rand(10,10),rand(10,10), rand(10,10)]
cmaps = ['viridis', 'binary', 'plasma', 'coolwarm', 'Greens', 'copper']


plt.figure(figsize=(10, 10))
for i in range(6):
    plt.subplot(3,2, i+1)
    plt.imshow(test_data[i], cmap=cmaps[i])
    plt.axis('off')
plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
plt.show()

In [ ]:
from numpy.random import rand
import matplotlib.pyplot as plt

test_data = [[rand(10,10), rand(10,10),rand(10,10)],[rand(10,10), rand(10,10),rand(10,10)],[rand(10,10), rand(10,10),rand(10,10)]]

columns = 2
rows = 3

figure, axarr = plt.subplots(rows,columns, figsize=(10, int(10*rows/columns)))

for i in range(rows):
    for j in range(columns):
        axarr[i, j].imshow(test_data[i][j])
        axarr[i, j].set_title('hola')
        axarr[i, j].axis(False)
plt.subplots_adjust(wspace=0, hspace=0.0675, left=0, right=1, bottom=0, top=1)
plt.show()

In [ ]:
import efficientnet.tfkeras as eftf
import tensorflow.keras.applications as tfka
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
ModelName = 'InceptionV3'
imageDimensions = (300,300,3)
nClasses = 1000
activation = 'softmax'
try:
    modelFunction = getattr(eftf, ModelName)
except:
    kerasApplications = [tfka.xception, tfka.vgg16, tfka.vgg19, tfka.resnet, tfka.resnet_v2, tfka.inception_v3,
                         tfka.inception_resnet_v2, tfka.mobilenet, tfka.densenet, tfka.nasnet, tfka.mobilenet_v2]
    for ka in kerasApplications:
        try:
            modelFunction = getattr(ka, ModelName)
            break
        except:
            pass
modelNoTop = modelFunction()  # 'imagenet')
modelNoTop.summary()
modelNoTop.save('model.h5')

In [ ]:

import tensorflow.keras.optimizers as tfko

modelNoTop.compile(loss='categorical_crossentropy',
              optimizer=tfko.Adam(learning_rate=0.001),
              metrics=['categorical_accuracy'])

In [ ]:
ClassNames = [2,3]
Mode = 'gfh'
monitor = ('binary_accuracy' if len(ClassNames) == 2 else 'categorical_accuracy') if Mode == 'classification' else 'val_iou_score'

In [ ]:
monitor

In [ ]:
print(modelNoTop.metrics_names)

In [ ]:
from six.moves import xrange
from math import ceil
from string import ascii_lowercase
#import keras_applications as ka
from collections import namedtuple
import functools
from os.path import exists
from warnings import warn
from tensorflow.python.keras import backend as _backend, layers as _layers, models as _models, utils as _utils
from tensorflow.keras.applications.imagenet_utils import preprocess_input as _preprocess_input
from tensorflow.keras.utils import get_custom_objects


_KERAS_BACKEND = None
_KERAS_LAYERS = None
_KERAS_MODELS = None
_KERAS_UTILS = None


backend = None
layers = None
models = None
keras_utils = None


DENSE_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 1. / 3.,
        'mode': 'fan_out',
        'distribution': 'uniform'
    }
}

CONV_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 2.0,
        'mode': 'fan_out',
        'distribution': 'normal'
    }
}

BlockArgs = namedtuple('BlockArgs', [
    'kernel_size', 'num_repeat', 'input_filters', 'output_filters',
    'expand_ratio', 'id_skip', 'strides', 'se_ratio'
])

DEFAULT_BLOCKS_ARGS = [
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16,
              expand_ratio=1, id_skip=True, strides=[1, 1], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=2, input_filters=16, output_filters=24,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=2, input_filters=24, output_filters=40,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=3, input_filters=40, output_filters=80,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=3, input_filters=80, output_filters=112,
              expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=4, input_filters=112, output_filters=192,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=192, output_filters=320,
              expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25)
]


def _obtain_input_shape(input_shape,
                        default_size,
                        min_size,
                        data_format,
                        require_flatten,
                        weights=None):
    if weights != 'imagenet' and input_shape and len(input_shape) == 3:
        if data_format == 'channels_first':
            if input_shape[0] not in {1, 3}:
                warn(
                    'This model usually expects 1 or 3 input channels. '
                    'However, it was passed an input_shape with ' +
                    str(input_shape[0]) + ' input channels.')
            default_shape = (input_shape[0], default_size, default_size)
        else:
            if input_shape[-1] not in {1, 3}:
                warn(
                    'This model usually expects 1 or 3 input channels. '
                    'However, it was passed an input_shape with ' +
                    str(input_shape[-1]) + ' input channels.')
            default_shape = (default_size, default_size, input_shape[-1])
    else:
        if data_format == 'channels_first':
            default_shape = (3, default_size, default_size)
        else:
            default_shape = (default_size, default_size, 3)
    if weights == 'imagenet' and require_flatten:
        if input_shape is not None:
            if input_shape != default_shape:
                raise ValueError('When setting `include_top=True` '
                                 'and loading `imagenet` weights, '
                                 '`input_shape` should be ' +
                                 str(default_shape) + '.')
        return default_shape
    if input_shape:
        if data_format == 'channels_first':
            if input_shape is not None:
                if len(input_shape) != 3:
                    raise ValueError(
                        '`input_shape` must be a tuple of three integers.')
                if input_shape[0] != 3 and weights == 'imagenet':
                    raise ValueError('The input must have 3 channels; got '
                                     '`input_shape=' + str(input_shape) + '`')
                if ((input_shape[1] is not None and input_shape[1] < min_size) or
                        (input_shape[2] is not None and input_shape[2] < min_size)):
                    raise ValueError('Input size must be at least ' +
                                     str(min_size) + 'x' + str(min_size) +
                                     '; got `input_shape=' +
                                     str(input_shape) + '`')
        else:
            if input_shape is not None:
                if len(input_shape) != 3:
                    raise ValueError(
                        '`input_shape` must be a tuple of three integers.')
                if input_shape[-1] != 3 and weights == 'imagenet':
                    raise ValueError('The input must have 3 channels; got '
                                     '`input_shape=' + str(input_shape) + '`')
                if ((input_shape[0] is not None and input_shape[0] < min_size) or
                        (input_shape[1] is not None and input_shape[1] < min_size)):
                    raise ValueError('Input size must be at least ' +
                                     str(min_size) + 'x' + str(min_size) +
                                     '; got `input_shape=' +
                                     str(input_shape) + '`')
    else:
        if require_flatten:
            input_shape = default_shape
        else:
            if data_format == 'channels_first':
                input_shape = (3, None, None)
            else:
                input_shape = (None, None, 3)
    if require_flatten:
        if None in input_shape:
            raise ValueError('If `include_top` is True, '
                             'you should specify a static `input_shape`. '
                             'Got `input_shape=' + str(input_shape) + '`')
    return input_shape


def get_submodules_from_kwargs(kwargs):
    backend = kwargs.get('backend', _KERAS_BACKEND)
    layers = kwargs.get('layers', _KERAS_LAYERS)
    models = kwargs.get('models', _KERAS_MODELS)
    utils = kwargs.get('utils', _KERAS_UTILS)
    for key in kwargs.keys():
        if key not in ['backend', 'layers', 'models', 'utils']:
            raise TypeError('Invalid keyword argument: %s', key)
    return backend, layers, models, utils


def inject_tfkeras_modules(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        kwargs['backend'] = _backend
        kwargs['layers'] = _layers
        kwargs['models'] = _models
        kwargs['utils'] = _utils
        return func(*args, **kwargs)
    return wrapper


def get_swish(**kwargs):
    backend, _, _, _ = get_submodules_from_kwargs(kwargs)

    def swish(x):
        return x * backend.sigmoid(x)

    return swish


def get_dropout(**kwargs):
    backend, layers, _, _ = get_submodules_from_kwargs(kwargs)

    class FixedDropout(layers.Dropout):
        def _get_noise_shape(self, inputs):
            if self.noise_shape is None:
                return self.noise_shape

            symbolic_shape = backend.shape(inputs)
            noise_shape = [symbolic_shape[axis] if shape is None else shape
                           for axis, shape in enumerate(self.noise_shape)]
            return tuple(noise_shape)

    return FixedDropout


def init_tfkeras_custom_objects():
    custom_objects = {
        'swish': inject_tfkeras_modules(get_swish)(),
        'FixedDropout': inject_tfkeras_modules(get_dropout)()
    }
    get_custom_objects().update(custom_objects)


def round_filters(filters, width_coefficient, depth_divisor):
    filters *= width_coefficient
    new_filters = int(filters + depth_divisor /
                      2) // depth_divisor * depth_divisor
    new_filters = max(depth_divisor, new_filters)
    if new_filters < 0.9 * filters:
        new_filters += depth_divisor
    return int(new_filters)


def mb_conv_block(inputs, block_args, activation, drop_rate=None, prefix='', ):

    has_se = (block_args.se_ratio is not None) and (
        0 < block_args.se_ratio <= 1)
    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

    Dropout = get_dropout(
        backend=backend,
        layers=layers,
        models=models,
        utils=keras_utils
    )

    filters = block_args.input_filters * block_args.expand_ratio
    if block_args.expand_ratio != 1:
        x = layers.Conv2D(filters, 1,
                          padding='same',
                          use_bias=False,
                          kernel_initializer=CONV_KERNEL_INITIALIZER,
                          name=prefix + 'expand_conv')(inputs)
        x = layers.BatchNormalization(
            axis=bn_axis, name=prefix + 'expand_bn')(x)
        x = layers.Activation(activation, name=prefix + 'expand_activation')(x)
    else:
        x = inputs

    x = layers.DepthwiseConv2D(block_args.kernel_size,
                               strides=block_args.strides,
                               padding='same',
                               use_bias=False,
                               depthwise_initializer=CONV_KERNEL_INITIALIZER,
                               name=prefix + 'dwconv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=prefix + 'bn')(x)
    x = layers.Activation(activation, name=prefix + 'activation')(x)

    if has_se:
        num_reduced_filters = max(1, int(
            block_args.input_filters * block_args.se_ratio
        ))
        se_tensor = layers.GlobalAveragePooling2D(
            name=prefix + 'se_squeeze')(x)

        target_shape = (1, 1, filters) if backend.image_data_format(
        ) == 'channels_last' else (filters, 1, 1)
        se_tensor = layers.Reshape(
            target_shape, name=prefix + 'se_reshape')(se_tensor)
        se_tensor = layers.Conv2D(num_reduced_filters, 1,
                                  activation=activation,
                                  padding='same',
                                  use_bias=True,
                                  kernel_initializer=CONV_KERNEL_INITIALIZER,
                                  name=prefix + 'se_reduce')(se_tensor)
        se_tensor = layers.Conv2D(filters, 1,
                                  activation='sigmoid',
                                  padding='same',
                                  use_bias=True,
                                  kernel_initializer=CONV_KERNEL_INITIALIZER,
                                  name=prefix + 'se_expand')(se_tensor)
        if backend.backend() == 'theano':
            pattern = ([True, True, True, False] if backend.image_data_format() == 'channels_last'
                       else [True, False, True, True])
            se_tensor = layers.Lambda(
                lambda x: backend.pattern_broadcast(x, pattern),
                name=prefix + 'se_broadcast')(se_tensor)
        x = layers.multiply([x, se_tensor], name=prefix + 'se_excite')

    x = layers.Conv2D(block_args.output_filters, 1,
                      padding='same',
                      use_bias=False,
                      kernel_initializer=CONV_KERNEL_INITIALIZER,
                      name=prefix + 'project_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=prefix + 'project_bn')(x)
    if block_args.id_skip and all(
            s == 1 for s in block_args.strides
    ) and block_args.input_filters == block_args.output_filters:
        if drop_rate < 1 and (drop_rate > 0):
            x = Dropout(drop_rate,
                        noise_shape=(None, 1, 1, 1),
                        name=prefix + 'drop')(x)
        x = layers.add([x, inputs], name=prefix + 'add')

    return x


def EfficientNet(width_coefficient,
                 depth_coefficient,
                 default_resolution,
                 dropout_rate=0.2,
                 drop_connect_rate=0.2,
                 depth_divisor=8,
                 blocks_args=DEFAULT_BLOCKS_ARGS,
                 model_name='efficientnet',
                 include_top=True,
                 weights='imagenet',
                 input_tensor=None,
                 input_shape=None,
                 pooling=None,
                 classes=1000,
                 **kwargs):
    global backend, layers, models, keras_utils
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    if not (weights in {'imagenet', 'noisy-student', None} or exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as `"imagenet"` with `include_top`'
                         ' as true, `classes` should be 1000')

    input_shape = _obtain_input_shape(input_shape,
                                      default_size=default_resolution,
                                      min_size=32,
                                      data_format=backend.image_data_format(),
                                      require_flatten=include_top,
                                      weights=weights)

    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if not backend.is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1
    activation = get_swish(**kwargs)

    x = img_input
    x = layers.Conv2D(round_filters(32, width_coefficient, depth_divisor), 3,
                      strides=(2, 2),
                      padding='same',
                      use_bias=False,
                      kernel_initializer=CONV_KERNEL_INITIALIZER,
                      name='stem_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='stem_bn')(x)
    x = layers.Activation(activation, name='stem_activation')(x)

    num_blocks_total = sum(block_args.num_repeat for block_args in blocks_args)
    block_num = 0
    for idx, block_args in enumerate(blocks_args):
        assert block_args.num_repeat > 0
        block_args = block_args._replace(
            input_filters=round_filters(block_args.input_filters,
                                        width_coefficient, depth_divisor),
            output_filters=round_filters(block_args.output_filters,
                                         width_coefficient, depth_divisor),
            num_repeat=int(ceil(depth_coefficient * block_args.num_repeat)))

        drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
        x = mb_conv_block(x, block_args,
                          activation=activation,
                          drop_rate=drop_rate,
                          prefix='block{}a_'.format(idx + 1))
        block_num += 1
        if block_args.num_repeat > 1:
            block_args = block_args._replace(
                input_filters=block_args.output_filters, strides=[1, 1])
            for bidx in xrange(block_args.num_repeat - 1):
                drop_rate = drop_connect_rate * \
                    float(block_num) / num_blocks_total
                block_prefix = 'block{}{}_'.format(
                    idx + 1,
                    ascii_lowercase[bidx + 1]
                )
                x = mb_conv_block(x, block_args,
                                  activation=activation,
                                  drop_rate=drop_rate,
                                  prefix=block_prefix)
                block_num += 1

    x = layers.Conv2D(round_filters(1024, width_coefficient, depth_divisor), 1,
                      padding='same',
                      use_bias=False,
                      kernel_initializer=CONV_KERNEL_INITIALIZER,
                      name='top_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='top_bn')(x)
    x = layers.Activation(activation, name='top_activation')(x)
    if include_top:
        x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        if dropout_rate and dropout_rate > 0:
            x = layers.Dropout(dropout_rate, name='top_dropout')(x)
        x = layers.Dense(classes,
                         activation='softmax',
                         kernel_initializer=DENSE_KERNEL_INITIALIZER,
                         name='probs')(x)
    else:
        if pooling == 'avg':
            x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        elif pooling == 'max':
            x = layers.GlobalMaxPooling2D(name='max_pool')(x)

    if input_tensor is not None:
        inputs = keras_utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input

    model = models.Model(inputs, x, name=model_name)

    return model


def EfficientNetL2(
        include_top=True,
        weights=None,
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=1024,
        **kwargs
):
    return EfficientNet(
        4.3, 5.3, 800, 0.5,
        model_name='efficientnet-l2',
        include_top=include_top, weights=weights,
        input_tensor=input_tensor, input_shape=input_shape,
        pooling=pooling, classes=classes,
        **kwargs
    )


setattr(EfficientNetL2, '__doc__', EfficientNet.__doc__)

EfficientNetL2 = inject_tfkeras_modules(EfficientNetL2)


def preprocess_input(x, **kwargs):
    kwargs = {k: v for k, v in kwargs.items() if k in [
        'backend', 'layers', 'models', 'utils']}
    return _preprocess_input(x, mode='torch', **kwargs)


preprocess_input = inject_tfkeras_modules(preprocess_input)

init_tfkeras_custom_objects()


In [ ]:
import efficientnet.tfkeras as eftf
import tensorflow.keras.applications as tfka
from segmentation_models import Unet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

ImageDimensions = (512,512,3)
nClasses = 1000

Mode = 'classification'
ModelName = 'EfficientNetL2'
Weights = None  # or 'imagenet'
activation = 'softmax'
if Mode == 'segmentation':
    DecoderBlock = 'upsampling'  # or 'transpose'

if Mode == 'classification':
    try:
        modelFunction = getattr(eftf, ModelName)
    except:
        try:
            modelFunction = EfficientNetL2
        except:
            kerasApplications = [tfka.xception, tfka.vgg16, tfka.vgg19, tfka.resnet, tfka.resnet_v2, tfka.inception_v3,
                                 tfka.inception_resnet_v2, tfka.mobilenet, tfka.densenet, tfka.nasnet, tfka.mobilenet_v2]
            for ka in kerasApplications:
                try:
                    modelFunction = getattr(ka, ModelName)
                    break
                except:
                    pass
    modelNoTop = modelFunction(input_shape=ImageDimensions,
                               classes=nClasses,
                               weights=Weights,
                               include_top=False,
                               pooling='avg')
else:
    modelNoTop = Unet(backbone_name=ModelName.lower(),
         input_shape=ImageDimensions,
         classes=nClasses,
         activation=activation,
         encoder_weights=Weights,
         decoder_block_type=DecoderBlock)
modelNoTop.summary()

In [ ]:

from tensorflow import image

In [ ]:
model = modelFunction(input_shape=ImageDimensions,
                               classes=nClasses,
                               weights=Weights)
model.summary()

In [ ]:
from glob import glob
from os.path import join

from tensorflow.python.keras.utils.data_utils import Sequence
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

from imgaug.augmenters import Sequential
from imgaug.augmenters.size import CenterPadToAspectRatio, Resize

from numpy import hsplit, asarray, array, floor, arange, zeros, uint8
from numpy.random import shuffle
from cv2 import imread, cvtColor, COLOR_BGR2RGB, IMREAD_UNCHANGED, INTER_AREA

import matplotlib.pyplot as plt

from cv2 import threshold, THRESH_BINARY
from numpy import argmax, around, newaxis, concatenate
from sklearn.metrics import confusion_matrix
from itertools import product

filenamesImagesTrain = list(
    glob(join('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\UNet\\EfficientNetB7\\Data', 'Train', 'Images', '*')))

In [ ]:
filenamesImagesTrain

In [ ]:
s = '1456208278. Hello world .start'
t1 = s.split() # ['1456208278', 'Hello', 'world', 'start']
t2 = t1[:-1] # ['Hello', 'world', 'start']
s2 = ''.join(t2)

In [ ]:
s2

In [ ]:
join('l[i]', ''.join((filenamesImagesTrain[4].split('\\')[-1]).split('.')[:-1])+'.png')

In [ ]:
import numpy as np
hola = np.array(filenamesImagesTrain).reshape((len(filenamesImagesTrain),1))

In [ ]:
hola.shape

In [ ]:
hola[0][0]

In [ ]:
adios = [[3,2,1,4],[2,4,2]]
sum([len(a) for a in adios])

In [ ]:
from glob import glob
from os.path import join

from tensorflow.python.keras.utils.data_utils import Sequence
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

from imgaug.augmenters import Sequential
from imgaug.augmenters.size import CenterPadToAspectRatio, Resize

from numpy import hsplit, asarray, array, floor, arange, zeros, uint8
from numpy.random import shuffle
from cv2 import imread, cvtColor, COLOR_BGR2RGB, IMREAD_UNCHANGED, INTER_AREA

import matplotlib.pyplot as plt

from cv2 import threshold, THRESH_BINARY
from numpy import argmax, around, newaxis, concatenate
from sklearn.metrics import confusion_matrix
from itertools import product


def load(relativePath, labels, n=None, validationPercentage=0.5):

    filenameImagesListTrain = []
    for l in labels:
        filenameImagesListTrain.append(
            list(glob(join(relativePath, 'Train', l, '*'))))

    filenameImagesListValidation = []
    for l in labels:
        filenameImagesListValidation.append(
            list(glob(join(relativePath, 'Validation', l, '*'))))

    if n is None:
        dataTrain = []
        for i in range(len(filenameImagesListTrain)):
            for j in range(len(filenameImagesListTrain[i])):
                dataTrain.append([filenameImagesListTrain[i][j], i])

        dataValidation = []
        for i in range(len(filenameImagesListValidation)):
            for j in range(len(filenameImagesListValidation[i])):
                dataValidation.append([filenameImagesListValidation[i][j], i])
    else:
        nTrain = int(n*(1.0-validationPercentage)/len(filenameImagesListTrain))
        nValidation = int(n*validationPercentage /
                          len(filenameImagesListValidation))

        dataTrain = []
        for i in range(nTrain):
            for j in range(len(filenameImagesListTrain)):
                dataTrain.append([filenameImagesListTrain[j][i], j])

        dataValidation = []
        for i in range(nValidation):
            for j in range(len(filenameImagesListValidation)):
                dataValidation.append([filenameImagesListValidation[j][i], j])

    shuffle(dataTrain)
    shuffle(dataValidation)

    def splitLabelsImages(data, labels):
        dataSplit = hsplit(asarray(data), array(range(1, 2)))
        images = dataSplit[0]
        labels = dataSplit[1]
        return images, labels

    trainImages, trainLabels = splitLabelsImages(dataTrain, labels)

    validationImages, validationLabels = splitLabelsImages(
        dataValidation, labels)

    return trainImages, trainLabels, validationImages, validationLabels

In [ ]:
import efficientnet.tfkeras as eftf
import tensorflow.keras.applications as tfka

from segmentation_models import Unet
from segmentation_models.losses import categorical_focal_dice_loss, binary_focal_dice_loss
from segmentation_models.metrics import IOUScore

from os.path import join

from tensorflow.keras.utils import to_categorical

import tensorflow.keras.optimizers as tfko
import imgaug.augmenters as iaa
import tensorflow.keras.callbacks as tfkc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import Utils.Callbacks as ca
from tensorflow.summary import create_file_writer, image
from os.path import exists
from os import mkdir
from datetime import datetime

In [ ]:
ImageDimensions = (512, 512, 3)

BatchSize = 1

Augmenters = iaa.Sequential(  # or None
    [
        iaa.Fliplr(0.5),
        iaa.Flipud(0.25),
        iaa.Sometimes(0.675,
                      [
                          iaa.Affine(scale={"x": (0.75, 1.25),
                                            "y": (0.75, 1.25)},
                                     cval=0,
                                     mode='constant')
                      ]
                      ),
        iaa.Sometimes(0.675,
                      [
                          iaa.Affine(translate_percent={
                              "x": (-0.25, 0.25),
                              "y": (-0.25, 0.25)},
                              mode='edge')
                      ]
                      ),
        iaa.Sometimes(0.675,
                      [
                          iaa.Multiply((0.75, 1.25),
                                       per_channel=0.25)
                      ]
                      ),
        iaa.Sometimes(0.675,
                      [
                          iaa.Pepper(p=(0.0, 0.125))
                      ]
                      ),
    ],
    random_order=True)

In [ ]:
ClassNames = ['neutral', 'smoke']

In [ ]:
Mode = 'classification'
if Mode == 'classification':
    from Utils.ImageClassification import load, DataGenerator, resizeImage, imageGrid, confusionMatrix
else:
    from Utils.ImageSegmentation import load, DataGenerator, resizeImage, imageGrid, confusionMatrix

In [ ]:
trainImages, trainLabels, validationImages, validationLabels = load('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\DataClassification', ClassNames)

In [ ]:
trainData = DataGenerator(trainImages,
                          trainLabels,
                          batchSize=BatchSize,
                          resize=resizeImage(ImageDimensions[:-1]),
                          augmenters=Augmenters,
                          shuffle=True)

validationData = DataGenerator(validationImages,
                               validationLabels,
                               batchSize=BatchSize,
                               resize=resizeImage(ImageDimensions[:-1]))

In [ ]:
from numpy import array
print(array(trainData[200][0]).shape)
print(array(trainData[200][1]).shape)

In [ ]:
print(array(trainData[200][0]))

In [ ]:
print(array(trainData[200][1]))

In [ ]:
import efficientnet.tfkeras as eftf
import tensorflow.keras.applications as tfka

from segmentation_models import Unet
from segmentation_models.losses import categorical_focal_dice_loss, binary_focal_dice_loss
from segmentation_models.metrics import IOUScore

from os.path import join

from tensorflow.keras.utils import to_categorical

import tensorflow.keras.optimizers as tfko
import imgaug.augmenters as iaa
import tensorflow.keras.callbacks as tfkc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import Utils.Callbacks as ca
from tensorflow.summary import create_file_writer, image
from os.path import exists
from os import mkdir
from datetime import datetime


Relative = join('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\Prueba')

RelativeData = join('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\DataClassification')

NamePrefix = 'Hola'


ClassNames = ['neutral', 'smoke']

ImageDimensions = (128, 128, 3)

BatchSize = 1

Augmenters = iaa.Sequential(  # or None
    [
        iaa.Fliplr(0.5),
        iaa.Flipud(0.25),
        iaa.Sometimes(0.675,
                      [
                          iaa.Affine(scale={"x": (0.75, 1.25),
                                            "y": (0.75, 1.25)},
                                     cval=0,
                                     mode='constant')
                      ]
                      ),
        iaa.Sometimes(0.675,
                      [
                          iaa.Affine(translate_percent={
                              "x": (-0.25, 0.25),
                              "y": (-0.25, 0.25)},
                              mode='edge')
                      ]
                      ),
        iaa.Sometimes(0.675,
                      [
                          iaa.Multiply((0.75, 1.25),
                                       per_channel=0.25)
                      ]
                      ),
        iaa.Sometimes(0.675,
                      [
                          iaa.Pepper(p=(0.0, 0.125))
                      ]
                      ),
    ],
    random_order=True)


LearningRate = 0.001

Optimizer = tfko.Adam(learning_rate=LearningRate)


SaveBest = True


Epochs = 4


'''
*****************************************************************************************************

---------------------------------------    Classification    ----------------------------------------

*****************************************************************************************************
*****************************************************************************************************

------------------------------------------    Imagenet    -------------------------------------------

*****************************************************************************************************
-----------------------------------------------------------------------------------------------------
Model               Size    Top-1 Accuracy  Top-5 Accuracy  Parameters  Depth   |   Segmentation    |
-----------------------------------------------------------------------------------------------------
*****************************************************************************************************
Xception           	88 MB	    0.790	        0.945	    22910480	126     |        ✗         |
-----------------------------------------------------------------------------------------------------
VGG16	            528 MB	    0.713       	0.901   	138357544	23      |        ✓         |
-----------------------------------------------------------------------------------------------------
VGG19	            549 MB	    0.713   	    0.900	    143667240	26      |        ✓         |
-----------------------------------------------------------------------------------------------------
ResNet50	        98 MB	    0.749	        0.921   	25636712	-       |        ✓         |
-----------------------------------------------------------------------------------------------------
ResNet101	        171 MB	    0.764	        0.928   	44707176	-       |        ✓         |
-----------------------------------------------------------------------------------------------------
ResNet152	        232 MB	    0.766	        0.931   	60419944	-       |        ✓         |
-----------------------------------------------------------------------------------------------------
ResNet50V2	        98 MB	    0.760	        0.930   	25613800	-       |        ✗         |
-----------------------------------------------------------------------------------------------------
ResNet101V2	        171 MB	    0.772	        0.938   	44675560	-       |        ✗         |
-----------------------------------------------------------------------------------------------------
ResNet152V2	        232 MB	    0.780	        0.942   	60380648	-       |        ✗         |
-----------------------------------------------------------------------------------------------------
InceptionV3	        92 MB	    0.779	        0.937   	23851784	159     |        ✓         |
-----------------------------------------------------------------------------------------------------
InceptionResNetV2	215 MB	    0.803	        0.953   	55873736	572     |        ✓         |
-----------------------------------------------------------------------------------------------------
MobileNet	        16 MB	    0.704	        0.895   	4253864	    88      |        ✓         |
-----------------------------------------------------------------------------------------------------
MobileNetV2     	14 MB	    0.713	        0.901   	3538984	    88      |        ✓         |
-----------------------------------------------------------------------------------------------------
DenseNet121     	33 MB	    0.750	        0.923   	8062504	    121     |        ✓         |
-----------------------------------------------------------------------------------------------------
DenseNet169     	57 MB	    0.762	        0.932   	14307880	169     |        ✓         |
-----------------------------------------------------------------------------------------------------
DenseNet201	        80 MB	    0.773	        0.936   	20242984	201     |        ✓         |
-----------------------------------------------------------------------------------------------------
NASNetMobile    	23 MB	    0.744	        0.919   	5326716	    -       |        ✗         |
-----------------------------------------------------------------------------------------------------
NASNetLarge	        343 MB	    0.825	        0.960   	88949818	-       |        ✗         |
-----------------------------------------------------------------------------------------------------
EfficientNetB0      21 MB       0.773           0.935       5330564     -       |        ✓         |
-----------------------------------------------------------------------------------------------------
EfficientNetB1      31 MB       0.792           0.945       7856232     -       |        ✓         |
-----------------------------------------------------------------------------------------------------
EfficientNetB2      36 MB       0.803           0.950       9177562     -       |        ✓         |
-----------------------------------------------------------------------------------------------------
EfficientNetB3      48 MB       0.817           0.956       12320528    -       |        ✓         |
-----------------------------------------------------------------------------------------------------
EfficientNetB4      75 MB       0.830           0.963       19466816    -       |        ✓         |
-----------------------------------------------------------------------------------------------------
EfficientNetB5      117 MB      0.837           0.967       30562520    -       |        ✓         |
-----------------------------------------------------------------------------------------------------
EfficientNetB6      166 MB      0.842           0.968       43265136    -       |        ✓         |
-----------------------------------------------------------------------------------------------------
EfficientNetB7      256 MB      0.844           0.971       66658680    -       |        ✓         |
-----------------------------------------------------------------------------------------------------
EfficientNetL2      1919 MB     0.885           0.987       478878100   -       |        ✗         |
-----------------------------------------------------------------------------------------------------
**************************************    No classification    **************************************
-----------------------------------------------------------------------------------------------------
ResNet18                                                                        |        ✓         |
-----------------------------------------------------------------------------------------------------
ResNet34                                                                        |        ✓         |
-----------------------------------------------------------------------------------------------------
SEResNet18                                                                      |        ✓         |
-----------------------------------------------------------------------------------------------------
SEResNet34                                                                      |        ✓         |
-----------------------------------------------------------------------------------------------------
SEResNet50                                                                      |        ✓         |
-----------------------------------------------------------------------------------------------------
SEResNet101                                                                     |        ✓         |
-----------------------------------------------------------------------------------------------------
SEResNet152                                                                     |        ✓         |
-----------------------------------------------------------------------------------------------------
ResNeXt50                                                                       |        ✓         |
-----------------------------------------------------------------------------------------------------
ResNeXt101                                                                      |        ✓         |
-----------------------------------------------------------------------------------------------------
SEResNeXt50                                                                     |        ✓         |
-----------------------------------------------------------------------------------------------------
SEResNeXt101                                                                    |        ✓         |
-----------------------------------------------------------------------------------------------------
SENet154                                                                        |        ✓         |
*****************************************************************************************************
-----------------------------------------------------------------------------------------------------
'''

Mode = 'classification'

ModelName = 'EfficientNetB0'

Weights = None  # or 'imagenet'

if Mode == 'segmentation':
    DecoderBlock = 'upsampling'  # or 'transpose'

'''
def LRSchedule(epoch):
    LearningRate = 0.001
    if epoch > 180:
        LearningRate *= 0.0005
    elif epoch > 160:
        LearningRate *= 0.001
    elif epoch > 120:
        LearningRate *= 0.01
    elif epoch > 80:
        LearningRate *= 0.1
    return LearningRate
'''

currentDir = join(Relative, NamePrefix+datetime.now().strftime('%Y%m%d%H%M%S'))
logDir = join(currentDir, 'Log')
monitor = ('val_binary_accuracy' if len(ClassNames) ==
           2 else 'val_categorical_accuracy') if Mode == 'classification' else 'val_iou_score'
checkpointsDir = join(currentDir, 'Checkpoints')

Callbacks = [
    tfkc.ModelCheckpoint(filepath=join(checkpointsDir,
                                       'Epoch{epoch:04d}Accuracy{'+monitor+':.4f}.h5'),
                         monitor=monitor,
                         mode='max',
                         save_weights_only=True,
                         save_best_only=SaveBest,
                         verbose=1),
    tfkc.ReduceLROnPlateau(monitor=monitor,
                           mode='max',
                           factor=0.5,
                           patience=4,
                           cooldown=4,
                           min_lr=0.0,
                           min_delta=0.0,
                           verbose=1),
    tfkc.ReduceLROnPlateau(monitor=monitor,
                           mode='max',
                           factor=0.5,
                           patience=16,
                           cooldown=0,
                           min_lr=0.0,
                           min_delta=0.0,
                           verbose=1),
    # tfkc.LearningRateScheduler(LRSchedule,
    #                           verbose=1),

]
'''
write_grads,
embedding_layer_names,
embeddings_data
'''
'''
----------------------------------------------------------------------------------------------
**********************************************************************************************
----------------------------------------------------------------------------------------------
'''

if not exists(currentDir):
    mkdir(currentDir)
if not exists(logDir):
    mkdir(logDir)
if not exists(checkpointsDir):
    mkdir(checkpointsDir)


if Mode == 'classification':
    from Utils.ImageClassification import load, DataGenerator, resizeImage, imageGrid, confusionMatrix
else:
    from Utils.ImageSegmentation import load, DataGenerator, resizeImage, imageGrid, confusionMatrix


trainImages, trainLabels, validationImages, validationLabels = load(
    RelativeData, ClassNames, n=256)


if Mode == 'classification':
    if len(ClassNames) <= 2:
        activation = 'sigmoid'
        nClasses = 1

        loss = 'binary_crossentropy'
        metrics = 'binary_accuracy'
    else:
        trainLabels = to_categorical(trainLabels)
        validationLabels = to_categorical(validationLabels)

        activation = 'softmax'
        nClasses = len(ClassNames)

        loss = 'categorical_crossentropy'
        metrics = 'categorical_accuracy'
else:
    nClasses = len(ClassNames)

    metrics = IOUScore(threshold=0.5)

    if nClasses == 1:
        activation = 'sigmoid'
        loss = binary_focal_dice_loss
    else:
        activation = 'softmax'
        loss = categorical_focal_dice_loss


trainData = DataGenerator(trainImages,
                          trainLabels,
                          batchSize=BatchSize,
                          resize=resizeImage(ImageDimensions[:-1]),
                          augmenters=Augmenters,
                          shuffle=True)

validationData = DataGenerator(validationImages,
                               validationLabels,
                               batchSize=BatchSize,
                               resize=resizeImage(ImageDimensions[:-1]))

if Mode == 'classification':
    try:
        modelFunction = getattr(eftf, ModelName)
    except:
        try:
            modelFunction = getattr(eftfl2, ModelName)
        except:
            kerasApplications = [tfka.xception, tfka.vgg16, tfka.vgg19, tfka.resnet, tfka.resnet_v2, tfka.inception_v3,
                                 tfka.inception_resnet_v2, tfka.mobilenet, tfka.densenet, tfka.nasnet, tfka.mobilenet_v2]
            for ka in kerasApplications:
                try:
                    modelFunction = getattr(ka, ModelName)
                    break
                except:
                    pass
    modelNoTop = modelFunction(input_shape=ImageDimensions,
                               classes=nClasses,
                               weights=Weights,
                               include_top=False,
                               pooling='avg')
    model = Sequential()
    model.add(modelNoTop)
    '''
    EfficientNet*   +Dropout
    VGG*            -AVG +Flatten +Dense +Dense
    MobileNet       +Reshape +Dropout +Conv2D +Reshape
    '''
    model.add(Dense(nClasses, activation=activation, name='predictions'))
else:
    model = Unet(backbone_name=ModelName.lower(),
                 input_shape=ImageDimensions,
                 classes=nClasses,
                 activation=activation,
                 encoder_weights=Weights,
                 decoder_block_type=DecoderBlock)
model.summary()

In [ ]:
model.save(join(currentDir, 'Model.h5'), include_optimizer=False)


model.compile(optimizer=Optimizer,
              loss=loss,
              metrics=[metrics])


history = model.fit(trainData,
                    validation_data=validationData,
                    epochs=Epochs,
                    callbacks=Callbacks)

In [ ]:
from os.path import join
from glob import glob
checkpointFilenames = list(glob(join('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\Humos\\Models\\EffB420200507125546', 'Checkpoints', '*')))
xMax = 1.0
for f in checkpointFilenames:
    x = float(f.split('\\')[-1][-9:-3])
    if  x < xMax:
        xMax = x
        checkpoint = f

In [ ]:
checkpoint

In [ ]:
from os import sep
print(sep)

In [ ]:
file = 'C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\Humos\\Models\\EffB420200507125546\\Checkpoints\\Epoch0001Accuracy0.5000.h5'

In [ ]:
file.split(sep)[-1]

In [ ]:
import numpy as np
data = [323,2]
print(np.array(data).astype(str))

In [ ]:
bool(True)

In [ ]:
from json import load
from os.path import join
from numpy import array

with open('Settings.json', 'r') as f:
    data = load(f)
    relativeData = join(str(data['RelativeData']))
    classNames = array(data['ClassNames']).astype(str)
    imageDimensions = array(data['ImageDimensions']).astype(int)
    mode = str(data['Mode'])
    modelName = str(data['ModelName'])
    weights = str(data['Weights'])
    decoderBlock = str(data['DecoderBlock'])
    batchSize = int(data['BatchSize'])
    epochs = int(data['Epochs'])

In [ ]:
print(weights)

In [ ]:
import efficientnet.tfkeras as eftf
import tensorflow.keras.applications as tfka

from segmentation_models import Unet

model = Unet(backbone_name='efficientnetb4',
             input_shape=(512,512,3),
             classes=1,
             activation='sigmoid',
             encoder_weights=None,
             decoder_block_type='transpose')
model.summary()

In [ ]:

model = Unet(backbone_name='efficientnetb4',
             input_shape=(512,512,3),
             classes=1,
             activation='sigmoid',
             encoder_weights=None,
             decoder_block_type='upsampling')
model.summary()

In [ ]:

model = Unet(backbone_name='seresnext101',
             input_shape=(512,512,3),
             classes=1,
             activation='sigmoid',
             encoder_weights=None,
             decoder_block_type='upsampling')
model.summary()

In [ ]:
import efficientnet.tfkeras as eftf
model = eftf.EfficientNetB4(input_shape=(32,32,3),
             classes=1,
            pooling='avg',
                               weights=None)
model.summary()

In [ ]:
from cv2 import imwrite, threshold, THRESH_BINARY, merge, bitwise_not, bitwise_and, bitwise_or, addWeighted, COLOR_RGB2BGR, INTER_AREA, COLOR_BGR2RGB
print(COLOR_RGB2BGR)

In [ ]:
a = 1, 2
import numpy as np
print(np.array(a).astype('uint8'))

In [ ]:
from numpy import array

from imgaug.augmenters.size import CenterPadToAspectRatio, Resize
from imgaug.augmenters import Sequential

from cv2 import imread, cvtColor, threshold, imwrite


maskSmoke = threshold(imread('C:\\Users\\mauro\\Documents\\Minsait\\OneDrive - Indra\\DeepLearning\\RaspberryPi\\Images\\smoke1.jpg', -1), 127, 255, 0)[1].astype('uint8')
maskNeutral = threshold(imread('C:\\Users\\mauro\\Documents\\Minsait\\OneDrive - Indra\\DeepLearning\\RaspberryPi\\Images\\smoke2.jpg', -1), 127, 255, 0)[1].astype('uint8')

print(sum(sum(maskNeutral)))
if sum(sum(maskSmoke)) <= 4 or sum(sum(maskNeutral)) > 4:
    print('Prediction FAILED!')
else:
    print('Prediction SUCCESSFUL!')

In [ ]:
from glob import glob
listFile = list(glob('C:\\Users\\mauro\\Documents\\Minsait\\OneDrive - Indra\\DeepLearning\\RaspberryPi\\Masks\\*'))
for f in listFile:
    maskNeutral = threshold(imread(f, -1), 127, 255, 0)[1].astype('uint8')
    print(sum(sum(maskNeutral)))

In [ ]:
11512*11512*255/0.324

In [ ]:

from cv2 import imread
from glob import glob
from numpy.random import shuffle
imageFilenames = list([3,12,42,12,123,4,125,2])
shuffle(imageFilenames)

In [ ]:
imageFilenames

In [ ]:
from six.moves import xrange
#x = range(4)
x = enumerate([234,324,65,675])
for i, r in x:
    print(i)
    print(r)

In [ ]:
from tensorflow.keras.layers import Conv2D, UpSampling2D, Concatenate, Activation, Conv2DTranspose, Input, MaxPooling2D
from tensorflow.keras.models import Model

inputshape = Input(shape=(512,512,3))

x = Conv2D(64,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification1a_conv')(inputshape)
x = Conv2D(64,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification1b_conv')(x)
skip1 = x
x = MaxPooling2D(pool_size=(2, 2),
             padding='same',
           name='classification1_maxp')(x)


x = Conv2D(128,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification2a_conv')(x)
x = Conv2D(128,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification2b_conv')(x)
skip2 = x
x = MaxPooling2D(pool_size=(2, 2),
             padding='same',
           name='classification2_maxp')(x)


x = Conv2D(256,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification3a_conv')(x)
x = Conv2D(256,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification3b_conv')(x)
skip3 = x
x = MaxPooling2D(pool_size=(2, 2),
             padding='same',
           name='classification3_maxp')(x)


x = Conv2D(512,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification4a_conv')(x)
x = Conv2D(512,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification4b_conv')(x)
skip4 = x
x = MaxPooling2D(pool_size=(2, 2),
             padding='same',
           name='classification4_maxp')(x)


x = Conv2D(1024,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='mida_conv')(x)
x = Conv2D(1024,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='midb_conv')(x)


x = UpSampling2D(size=2, name='decoder4_upsampling')(x)
x = Concatenate(axis=3, name='decoder4_concat')([x, skip4])
x = Conv2D(512,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='decoder4a_conv')(x)
x = Conv2D(512,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='decoder4b_conv')(x)


x = UpSampling2D(size=2, name='decoder3_upsampling')(x)
x = Concatenate(axis=3, name='decoder3_concat')([x, skip3])
x = Conv2D(256,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='decoder3a_conv')(x)
x = Conv2D(256,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='decoder3b_conv')(x)


x = UpSampling2D(size=2, name='decoder2_upsampling')(x)
x = Concatenate(axis=3, name='decoder2_concat')([x, skip2])
x = Conv2D(128,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='decoder2a_conv')(x)
x = Conv2D(128,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='decoder2b_conv')(x)


x = UpSampling2D(size=2, name='decoder1_upsampling')(x)
x = Concatenate(axis=3, name='decoder1_concat')([x, skip1])
x = Conv2D(64,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='decoder1a_conv')(x)
x = Conv2D(64,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='decoder1b_conv')(x)


x = Conv2D(filters=1,
        kernel_size=1,
        padding='same',
        name='final_conv')(x)
x = Activation('sigmoid',
               name='sigmoid')(x)


model = Model(inputshape, x, name='U-Net')
model.summary()

In [ ]:
from tensorflow.keras.layers import Conv2D, UpSampling2D, Concatenate, Activation, Conv2DTranspose, Input, MaxPooling2D
from tensorflow.keras.models import Model

inputshape = Input(shape=(512,512,3))

x = Conv2D(64,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification1a_conv')(inputshape)
x = Conv2D(64,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification1b_conv')(x)
skip1 = x
x = MaxPooling2D(pool_size=(2, 2),
             padding='same',
           name='classification1_maxp')(x)


x = Conv2D(128,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification2a_conv')(x)
x = Conv2D(128,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification2b_conv')(x)
skip2 = x
x = MaxPooling2D(pool_size=(2, 2),
             padding='same',
           name='classification2_maxp')(x)


x = Conv2D(256,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification3a_conv')(x)
x = Conv2D(256,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification3b_conv')(x)
skip3 = x
x = MaxPooling2D(pool_size=(2, 2),
             padding='same',
           name='classification3_maxp')(x)


x = Conv2D(512,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification4a_conv')(x)
x = Conv2D(512,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='classification4b_conv')(x)
skip4 = x
x = MaxPooling2D(pool_size=(2, 2),
             padding='same',
           name='classification4_maxp')(x)


x = Conv2D(1024,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='mida_conv')(x)
x = Conv2D(1024,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='midb_conv')(x)


x = Conv2DTranspose(512,
                    kernel_size=(4, 4),
                    strides=(2, 2),
                    padding='same',
                    name='decoder4_transpose')(x)
x = Concatenate(axis=3, name='decoder4_concat')([x, skip4])
x = Conv2D(512,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='decoder4_conv')(x)


x = Conv2DTranspose(256,
                    kernel_size=(4, 4),
                    strides=(2, 2),
                    padding='same',
                    name='decoder3_transpose')(x)
x = Concatenate(axis=3, name='decoder3_concat')([x, skip3])
x = Conv2D(256,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='decoder3_conv')(x)


x = Conv2DTranspose(128,
                    kernel_size=(4, 4),
                    strides=(2, 2),
                    padding='same',
                    name='decoder2_transpose')(x)
x = Concatenate(axis=3, name='decoder2_concat')([x, skip2])
x = Conv2D(128,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='decoder2_conv')(x)


x = Conv2DTranspose(64,
                    kernel_size=(4, 4),
                    strides=(2, 2),
                    padding='same',
                    name='decoder1_transpose')(x)
x = Concatenate(axis=3, name='decoder1_concat')([x, skip1])
x = Conv2D(64,
           3,
           strides=[1, 1],
           activation='relu',
           padding='same',
           name='decoder1_conv')(x)


x = Conv2D(filters=1,
        kernel_size=1,
        padding='same',
        name='final_conv')(x)
x = Activation('sigmoid',
               name='sigmoid')(x)


model = Model(inputshape, x, name='U-Net')
model.summary()

In [ ]:

print(round_filters(16, 1.4, 8)*6)

In [ ]:
from os.path import join, exists
from os import sep, mkdir
from glob import glob
from numpy import hsplit, asarray, array, floor, arange, zeros, uint8, around, newaxis, concatenate, float32
from numpy.random import shuffle, seed

from imgaug.augmenters import Sequential
from imgaug.augmenters.size import CenterPadToAspectRatio, Resize
from cv2 import INTER_AREA, imread, cvtColor, COLOR_BGR2RGB, IMREAD_UNCHANGED, imwrite, threshold, THRESH_BINARY, merge, bitwise_not, bitwise_and, bitwise_or, addWeighted, COLOR_RGB2BGR

from tensorflow.python.keras.utils.data_utils import Sequence
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

import matplotlib.pyplot as plt

seed(234456)

def load(relativePath, labels, n=None, validationPercentage=0.5):

    filenameImagesListTrain = []
    for l in labels:
        filenameImagesListTrain.append(
            list(glob(join(relativePath, 'Train', l, '*'))))

    filenameImagesListValidation = []
    for l in labels:
        filenameImagesListValidation.append(
            list(glob(join(relativePath, 'Validation', l, '*'))))

    if n is None:
        dataTrain = []
        for i in range(len(filenameImagesListTrain)):
            for j in range(len(filenameImagesListTrain[i])):
                dataTrain.append([filenameImagesListTrain[i][j], i])

        dataValidation = []
        for i in range(len(filenameImagesListValidation)):
            for j in range(len(filenameImagesListValidation[i])):
                dataValidation.append([filenameImagesListValidation[i][j], i])
    else:
        nTrain = int(n*(1.0-validationPercentage)/len(filenameImagesListTrain))
        nValidation = int(n*validationPercentage /
                          len(filenameImagesListValidation))

        dataTrain = []
        for i in range(nTrain):
            for j in range(len(filenameImagesListTrain)):
                dataTrain.append([filenameImagesListTrain[j][i], j])

        dataValidation = []
        for i in range(nValidation):
            for j in range(len(filenameImagesListValidation)):
                dataValidation.append([filenameImagesListValidation[j][i], j])


    def splitLabelsImages(data, labels):
        dataSplit = hsplit(asarray(data), array(range(1, 2)))
        images = dataSplit[0]
        labels = dataSplit[1]
        return images, labels

    trainImages, trainLabels = splitLabelsImages(dataTrain, labels)

    validationImages, validationLabels = splitLabelsImages(
        dataValidation, labels)

    return trainImages, trainLabels, validationImages, validationLabels


def resize(imageDimensions):
    return Sequential(
        [
            CenterPadToAspectRatio(float(imageDimensions[0])/float(imageDimensions[1]),
                                   pad_mode='constant',
                                   pad_cval=0),
            Resize(imageDimensions,
                   interpolation=INTER_AREA)
        ])

def imageGrid(images, labels, classNames=['0'], perRow=2, imageDimensions=(64, 64), nImages=16):
    nClasses = len(classNames)+1
    columns = perRow
    rows = int(nImages/perRow)
    figure, axarr = plt.subplots(rows, columns, figsize=(
        imageDimensions[0], int(imageDimensions[1]*rows/columns)))
    k = 0
    for i in range(rows):
        for j in range(perRow):
            k = int(j)+(i*perRow)
            axarr[i, j].imshow(images[k])
            axarr[i, j].set_title(classNames[int(labels[k])], fontsize=256)
            axarr[i, j].axis(False)
    plt.subplots_adjust(wspace=0, hspace=0.0675,
                        left=0, right=1, bottom=0, top=1)
    return figure

def prepareLabels(indexes, imagesLabels):
    imagesLabelsList = []
    for k in range(len(indexes)):
        img = None
        for i in range(len(imagesLabels)):
            imgLabel = imagesLabels[i][k]
            if img is None:
                img = zeros(
                    (imgLabel.shape[0], imgLabel.shape[1], len(imagesLabels)), uint8)
            for x in range(len(imgLabel)):
                for y in range(len(imgLabel[x])):
                    img[x][y][i] = imgLabel[x][y]
        imagesLabelsList.append(img/255.0)
    return array(imagesLabelsList).astype('float32')

import imgaug.augmenters as iaa
augmenters =iaa.Sequential(  # iaa.Sequential() or None
    [
        iaa.Fliplr(0.5),
        iaa.Flipud(0.5),
        iaa.Multiply((0.25, 1.75),
                     per_channel=False),
        iaa.Multiply((0.75, 1.125),
                     per_channel=0.25),
        iaa.Affine(scale={'x': (0.5, 1.5),
                          'y': (0.5, 1.5)},
                   cval=0,
                   mode='constant'),
        iaa.Affine(translate_percent={
            'x': (-0.5, 0.5),
            'y': (-0.5, 0.5)},
            cval=0,
            mode='constant')
    ],
    random_order=False)

imagePaths, labels, _, _ = load('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\Humos\\Data\\Fig', ['neutral', 'smoke'])

images = [cvtColor(imread(str(k[0])), COLOR_BGR2RGB) for k in imagePaths]

resizeR = resize([512,512])

images = resizeR.augment_images(images)

images = augmenters.augment_images(images)

figur = imageGrid(images, labels, classNames=['neutral', 'smoke'], perRow=2, imageDimensions=(128, 128), nImages=4)
figur.show()

In [4]:
from os.path import join, exists
from os import sep, mkdir
from glob import glob
from numpy import hsplit, asarray, array, floor, arange, zeros, uint8, around, newaxis, concatenate, float32, argmax
from numpy.random import shuffle, seed

from imgaug.augmenters import Sequential
from imgaug.augmenters.size import CenterPadToAspectRatio, Resize
from cv2 import INTER_AREA, imread, cvtColor, COLOR_BGR2RGB, IMREAD_UNCHANGED, imwrite, threshold, THRESH_BINARY, merge, bitwise_not, bitwise_and, bitwise_or, addWeighted, COLOR_RGB2BGR

from tensorflow.python.keras.utils.data_utils import Sequence
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model

def load(relativePath, labels, n=None, validationPercentage=0.5):

    filenameImagesListTrain = []
    for l in labels:
        filenameImagesListTrain.append(
            list(glob(join(relativePath, 'Train', l, '*'))))

    filenameImagesListValidation = []
    for l in labels:
        filenameImagesListValidation.append(
            list(glob(join(relativePath, 'Validation', l, '*'))))

    if n is None:
        dataTrain = []
        for i in range(len(filenameImagesListTrain)):
            for j in range(len(filenameImagesListTrain[i])):
                dataTrain.append([filenameImagesListTrain[i][j], i])

        dataValidation = []
        for i in range(len(filenameImagesListValidation)):
            for j in range(len(filenameImagesListValidation[i])):
                dataValidation.append([filenameImagesListValidation[i][j], i])
    else:
        nTrain = int(n*(1.0-validationPercentage)/len(filenameImagesListTrain))
        nValidation = int(n*validationPercentage /
                          len(filenameImagesListValidation))

        dataTrain = []
        for i in range(nTrain):
            for j in range(len(filenameImagesListTrain)):
                dataTrain.append([filenameImagesListTrain[j][i], j])

        dataValidation = []
        for i in range(nValidation):
            for j in range(len(filenameImagesListValidation)):
                dataValidation.append([filenameImagesListValidation[j][i], j])

    shuffle(dataTrain)
    shuffle(dataValidation)

    def splitLabelsImages(data, labels):
        dataSplit = hsplit(asarray(data), array(range(1, 2)))
        images = dataSplit[0]
        labels = dataSplit[1]
        return images, labels

    trainImages, trainLabels = splitLabelsImages(dataTrain, labels)

    validationImages, validationLabels = splitLabelsImages(
        dataValidation, labels)

    return trainImages, trainLabels, validationImages, validationLabels


def resize(imageDimensions):
    return Sequential(
        [
            CenterPadToAspectRatio(float(imageDimensions[0])/float(imageDimensions[1]),
                                   pad_mode='constant',
                                   pad_cval=0),
            Resize(imageDimensions,
                   interpolation=INTER_AREA)
        ])


def imageGrid(images, labels, classNames=['0'], perRow=4, imageDimensions=(64, 64), nImages=16):
    rows = int(nImages/perRow)
    figure, axarr = plt.subplots(rows, perRow, figsize=(
        imageDimensions[0], int(imageDimensions[1]*rows/perRow)))
    k = 0
    for i in range(rows):
        for j in range(perRow):
            k = int(j)+(i*perRow)
            axarr[i, j].imshow(resize(imageDimensions).augment_image(
                cvtColor(imread(images[k]), COLOR_BGR2RGB)))
            #axarr[i, j].set_title(classNames[int(labels[k])], fontsize=256)
            axarr[i, j].axis(False)
    plt.subplots_adjust(wspace=0, hspace=0.125,
                        left=0, right=1, bottom=0, top=1)
    return figure

from tensorflow import nn
from tensorflow.keras.backend import shape
from tensorflow.keras.layers import Dropout


class FixedDropout(Dropout):
    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape
        return tuple([shape(inputs)[i] if sh is None else sh for i, sh in enumerate(self.noise_shape)])


customObjects = {
    'swish': nn.swish,
    'FixedDropout': FixedDropout
}

imagePaths = list(glob(join('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\Humos\\Prediction', 'Prueba', '*')))


figur = imageGrid(imagePaths, [], classNames=['smoke'], perRow=2, imageDimensions=(128, 128), nImages=8)
figur.show()

C:\Users\mauro\.conda\envs\tf-cv-ft-gpu\lib\site-packages\ipykernel_launcher.py:120: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.


In [2]:
model = load_model(join('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\Humos\\Prediction\\Models', 'InceptionResNetV2'+'.h5'), compile=False, custom_objects=customObjects)

label = []
for i in range(len(imagePaths)):
    image = cvtColor(imread(imagePaths[i]), COLOR_BGR2RGB)

    imageResized = resize([int(model.input.shape[1]), int(model.input.shape[2])]).augment_image(image)

    imagePredicted = array([imageResized/255.0]).astype(float32)

    prediction = model.predict(imagePredicted, batch_size=1)

    label.append(0 if prediction[0][0] < 0.5 else 1)

figur = imageGrid(imagePaths, label, classNames=['neutral', 'smoke'], perRow=2, imageDimensions=(128, 128), nImages=8)
figur.show()

C:\Users\mauro\.conda\envs\tf-cv-ft-gpu\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  app.launch_new_instance()


In [ ]:
model = load_model(join('C:\\Users\\mauro\\Documents\\Minsait\\NoOneDrive\\Humos\\Prediction\\Models', 'ResNet152V2'+'.h5'), compile=False, custom_objects=customObjects)

label = []
for i in range(len(imagePaths)):
    image = cvtColor(imread(imagePaths[i]), COLOR_BGR2RGB)

    imageResized = resize([int(model.input.shape[1]), int(model.input.shape[2])]).augment_image(image)

    imagePredicted = array([imageResized/255.0]).astype(float32)

    prediction = model.predict(imagePredicted, batch_size=1)

    label.append(0 if prediction[0][0] < 0.5 else 1)

figur = imageGrid(imagePaths, label, classNames=['neutral', 'smoke'], perRow=2, imageDimensions=(128, 128), nImages=8)
figur.show()